<a href="https://colab.research.google.com/github/waytoakash/waytoakash/blob/main/RP_Sales_Master_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**⏯Import Libraries**

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
import re
import time

from dateutil.relativedelta import relativedelta #
from functools import reduce

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

import gspread
from google.colab import auth
auth.authenticate_user() #

from oauth2client.client import GoogleCredentials #
#gc = gspread.authorize(GoogleCredentials.get_application_default()) #

auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

from gspread_dataframe import set_with_dataframe

!pip install fuzzywuzzy
!pip install python-Levenshtein

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def changeDateFormat(row):
  return pd.to_datetime(row, errors='coerce').dt.date

def changeDateTimeFormat(row):
  return pd.to_datetime(row, errors='coerce')

def cleanText(row):
  return row.str.upper().str.strip()

def properText(row):
  return row.str.title().str.strip()

def lowerText(row):
  return row.str.lower().str.strip()

def get_data_df(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[1:], columns = l[0])
  return ldf

def getDump(mastermerge, dump_name,start_date, end_date):
  start_date = pd.to_datetime(start_date).date()
  end_date = pd.to_datetime(end_date).date()
  file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/"
  reg = re.compile(".+" + dump_name+".csv")

  for (root, dirs, files) in os.walk(file_path):
      for file in files:
        path = os.path.join(root, file)
        date = str(path).split('/')[7:][0]
        date = pd.to_datetime(date).date()

        if bool(re.match(reg, path)) & (date >= start_date) & (date <= end_date):
          temp = pd.read_csv(path,parse_dates=True)
          temp['DumpDate'] = date
          read_dump_name = str(path).split('/')[9:][0]
          #print(read_dump_name + " on Date:: " + str(date) +  " Dump of shape:: " + str(temp.shape) + " read from path:: "+ path)
          mastermerge = mastermerge.append(temp)

  mastermerge[['DumpDate']] = mastermerge[['DumpDate']].apply(changeDateFormat)
  return mastermerge

Mounted at /content/drive


In [ ]:
daysDelta = 0
DumpDate =  pd.to_datetime((datetime.today() - timedelta(days = daysDelta)).date()).strftime('%d %B %Y')

Last30DaysDate = (datetime.today() - timedelta(days = 30 + daysDelta)).date()
Last60DaysDate = (datetime.today() - timedelta(days = 60 + daysDelta)).date()
Last90DaysDate = (datetime.today() - timedelta(days = 90 + daysDelta)).date()
Last7DaysDate = (datetime.today() - timedelta(days = 7 + daysDelta)).date()
Last45DaysDate = (datetime.today() - timedelta(days = 45 + daysDelta)).date()
MonthStartDate = datetime.today().date().replace(day=1)

start_date = Last30DaysDate.strftime('%d %B %Y')

file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/Mar 2025/"


#**HR data**

In [ ]:
def get_data(spreadSheetId, rangeName):

    wb = gc.open_by_key(spreadSheetId)
    ws = wb.worksheet(rangeName)
    rows = ws.get_all_values()
    return rows
#get all data in a sheet as df and there 1st row things as the df labels.
#If sheets don't have their labels in the first row than we have to do it manuaaly as per the data
def get_data_df(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[1:], columns = l[0])
  return ldf

spreadSheetId="1QLOS849fPNPVYYGND7MmnMivVf4o1VZEhLhJS89a17I"
rangeName="HR"
HR=get_data_df(spreadSheetId, rangeName)


HR = HR[['Emp Code', 'Official Email ID','Employee Name','Location', 'Department', 'Designation','Role','Reporting Supervisor E-mail ID',
         'Status', 'DOJ', 'Reporting Date','LWD']]

HR.rename({'Official Email ID':'Emp Email ID','Location':'Emp City', 'Reporting Supervisor E-mail ID':'Manager Email ID',
         'Status':'Emp Status', 'Reporting Date':'DOR'}, inplace = True, axis = 1)
HR = HR[~(HR['Emp Code'].isna())]
HR = HR[HR['Emp Code'] != '']
HR[['DOJ','DOR','LWD']] = HR[['DOJ','DOR','LWD']].apply(changeDateFormat)
HR[['Emp Code','Role']] = HR[['Emp Code','Role']].apply(cleanText)
HR['Emp Email ID'] = HR['Emp Email ID'].str.strip().str.lower()
HR.loc[HR['Emp City'] == 'Bengaluru','Emp City'] = 'Bangalore'

HR_Sales = HR[(HR['Department'] == "Sales") | (HR['Emp Email ID']=='somsi.jaiswal@nobroker.in')]
HR_Sales = HR_Sales[HR_Sales['Emp Email ID']!=""]

#**📕Read RAW data**

-----------

In [ ]:
spreadSheetId="1D1LZEYnDNa9aYYr33qbtyZylKUIYWX4RJHtIz2KYqZY"
rangeName="Form Responses 1"
closuresRaw=get_data_df(spreadSheetId, rangeName)

closuresRaw.rename({'Timestamp':'Closure Form Filled Date', 'Email Address':'Closure Form Filled By', 'Please select the Closure type':'Closure Type', 'Uniview Closure Number (Current)':'Closure Number',
        'Specify the HANDOVER MEETING ID':'HO Meeting ID', 'Society Name':'Society Name (Sales)','Property Managed by or Signed by':'Property Type', 'Closure date':'Closure Date','VMS Trial Period':'VMSTrialPeriod',
        'ERP Trial Period':'ERPTrialPeriod','Occupancy Percentage (Specify the percentage between 0% - 100%)':'Occupancy Percentage', 'Product(s) Sold':'Sales Product', 'BD Signed - Primary Email':'Closure Done By (Primary Email)',
        'Subscription Revenue (Annual - Excluding GST)':'Subscription Revenue','RP (Revenue Points) to be claimed as per calculator':'RP (Revenue Points) to be claimed','Revenue to be claimed as per calculator':'Revenue to be claimed',
        'BD Signed - Secondary Email (if applicable)':'Closure Done By (Secondary Email)','Promised Monetisation Revenue to society per annum in ₹ (Specify 0 if monetisation has not promised)':'Promised Monetisation Revenue per Annum',
        'Total Constructed and Sold flats ready to move in':'Total Flats','Please upload relevant closure proof document(s) ':'Closure Proof Link','Closure Agreement Type':'Agreement Type',
        'Promised Monetisation Revenue to society per annum in ₹ (Specify 0 if monetisation has not promised)':'Promised Mntz Revenue per annum to society'}, inplace = True, axis = 1)


closuresRaw[['Closure Form Filled Time']] = closuresRaw[['Closure Form Filled Date']].apply(changeDateTimeFormat)
closuresRaw[['Closure Date','Closure Form Filled Date']] = closuresRaw[['Closure Date','Closure Form Filled Date']].apply(changeDateFormat)
closuresRaw[['Society Name (Sales)']] = closuresRaw[['Society Name (Sales)']].apply(cleanText)
closuresRaw = closuresRaw[(closuresRaw['Closure Date'] >= pd.to_datetime("2023-10-01").date())]

closuresRaw.loc[closuresRaw['Closure Type'] == 'New Closure', 'Original Closure Number'] = closuresRaw['Closure Number']

closuresRaw = closuresRaw[['Closure Type','Closure Date','Original Closure Number','Closure Number','Closure Form Filled Date', 'Closure Form Filled Time',
                      'Closure Form Filled By','Team Type', 'Lead Type', 'Closure Done By (Primary Email)','Closure Done By (Secondary Email)',
                      'Society Type', 'Property Type','VMSTrialPeriod','ERPTrialPeriod','HO Meeting ID','Society Name (Sales)','Closure City',
                      'Sales Product', 'Total Flats','VMS Sold Flats','ERP Sold Flats','Occupancy Percentage','Pin Code','Society Latitude', 'Society Longitude',
                      'Rent of the society','Brand New Devices','New Devices','Refurbished Devices','Subscription Revenue','Upfront Revenue (One time)','Access control/Boom Barrier net revenue',
                      'Welfare Amount','Monetization Advance Amount','RP (Revenue Points) to be claimed','Revenue to be claimed','Current VMS software used',	'Current ERP software used',
                      'Closure Proof Link','Agreement Type','Competition Proof Link (VMS)','Competition Proof Link (ERP)','Promised Mntz Revenue per annum to society',
                      'VMS Paid Subscription Tenure','ERP Paid Subscription Tenure','Biometrics Devices','Hood Rev Code']]

############################################

closuresRaw['Closure Done By (Primary Email)'] = closuresRaw['Closure Done By (Primary Email)'].str.strip().str.lower()
closuresRaw['Closure Done By (Secondary Email)'] = closuresRaw['Closure Done By (Secondary Email)'].str.strip().str.lower()

closuresRaw.loc[closuresRaw['Sales Product'] == 'ERP only', 'VMS Sold Flats'] = 0
closuresRaw.loc[closuresRaw['Sales Product'] == 'VMS only', 'ERP Sold Flats'] = 0

closuresRaw.loc[closuresRaw['Sales Product'] == 'ERP only', 'VMSTrialPeriod'] = ''
closuresRaw.loc[closuresRaw['Sales Product'] == 'VMS only', 'ERPTrialPeriod'] = ''

closuresRaw['ERP Sold Flats'] = pd.to_numeric(closuresRaw['ERP Sold Flats'], errors = 'coerce')
closuresRaw['VMS Sold Flats'] = pd.to_numeric(closuresRaw['VMS Sold Flats'], errors = 'coerce')

closuresRaw['Sold Flats'] = closuresRaw[['ERP Sold Flats','VMS Sold Flats']].max(axis=1)

#############################################
#Alerts

closuresRaw = pd.merge(closuresRaw, HR_Sales[['Emp Email ID','Emp City','Role','Designation','Manager Email ID']], left_on = 'Closure Done By (Primary Email)', right_on = 'Emp Email ID', how = 'left')
closuresRaw.rename({'Emp City':'Primary BD City','Role':'Primary BD Role', 'Designation':'Primary BD Designation','Manager Email ID':'Primary RM Mail'}, inplace = True, axis = 1)
closuresRaw.drop({'Emp Email ID'}, inplace = True, axis = 1)

closuresRaw.loc[closuresRaw['Primary BD Role'].isna(),'WrongPrimaryBDMail'] = True

closuresRaw = pd.merge(closuresRaw, HR_Sales[['Emp Email ID','Emp City','Role','Designation','Manager Email ID']], left_on = 'Closure Done By (Secondary Email)', right_on = 'Emp Email ID', how = 'left')
closuresRaw.rename({'Emp City':'Secondary BD City','Role':'Secondary BD Role', 'Designation':'Secondary BD Designation','Manager Email ID':'Secondary RM Mail'}, inplace = True, axis = 1)
closuresRaw.drop({'Emp Email ID'}, inplace = True, axis = 1)

closuresRaw.loc[(closuresRaw['Secondary BD Role'].isna()) & (closuresRaw['Closure Done By (Secondary Email)'] != ""),'WrongSecondaryBDMail'] = True

###########################
# closuresRaw.drop_duplicates(inplace=True)

#closuresRaw = closuresRaw.sort_values(['Closure Date'], ascending = True).groupby(['Original Closure Number','Closure Number']).head(5)

closuresRawDuplicateClosureNumber = closuresRaw[closuresRaw['Closure Number'].duplicated()]['Closure Number'].drop_duplicates()
closuresRaw.loc[closuresRaw['Closure Number'].isin(closuresRawDuplicateClosureNumber),'DuplicateClosureNumber'] = True

###########################

closuresRaw.loc[:,'Closure Form Filling TAT'] = (closuresRaw['Closure Form Filled Date'] - closuresRaw['Closure Date']).apply(lambda x: x.days)

###########################

closuresRaw.loc[:,'CrossCityClosure'] = (closuresRaw['Closure City'] != closuresRaw['Primary BD City']) | ( (~closuresRaw['Secondary BD City'].isna()) & (closuresRaw['Closure City'] != closuresRaw['Secondary BD City']))
closuresRaw.loc[closuresRaw['CrossCityClosure'] == False,'CrossCityClosure'] = ""
###########################

closuresRaw.loc[(closuresRaw['Closure Type'].isin({'Upsell (VMS/ERP)','Winback'})) & (closuresRaw['Closure Number'] == closuresRaw['Original Closure Number']),'WrongOriginalClosureNumber'] = True

###########################

closuresRawActivity = closuresRaw[closuresRaw['Closure Type'].isin({'Upsell (VMS/ERP)','Winback'})]

closuresRawActivity = closuresRaw.sort_values(['Closure Date'], ascending = True).groupby(['Original Closure Number']).head(2)
closuresRawActivity = closuresRawActivity[(closuresRawActivity['Original Closure Number'].duplicated(keep= False)) & (closuresRawActivity['Original Closure Number']!= "")]

closuresRawActivity.loc[:, 'Closure Month'] = pd.to_datetime(closuresRawActivity['Closure Date']).dt.month

closuresRawActivity['Months Since Last Closure'] = closuresRawActivity.groupby(['Original Closure Number'])['Closure Month'].transform(lambda x: x.diff())

closuresRawActivity['Days Since Last Closure'] = closuresRawActivity.groupby(['Original Closure Number'])['Closure Date'].transform(lambda x: x.diff().dt.days)

closuresRawActivity.sort_index(inplace=True)


closuresRawActivity.loc[closuresRawActivity['Days Since Last Closure']<=90, 'ClosureActivity'] = 'Within 90 days'
closuresRawActivity.loc[closuresRawActivity['Days Since Last Closure']<=35, 'ClosureActivity'] = 'Within 35 days'
closuresRawActivity.loc[closuresRawActivity['Months Since Last Closure']==0, 'ClosureActivity'] = 'Within Month Closure Activity'

#closuresRawActivity = closuresRawActivity[closuresRawActivity['ClosureActivity'].notna()][['Original Closure Number','Closure Number','ClosureActivity']].drop_duplicates() old code changed as .notna() was not being read.

closuresRawActivity = closuresRawActivity[closuresRawActivity['ClosureActivity']!='nan'][['Original Closure Number','Closure Number','ClosureActivity']].drop_duplicates()

closuresRaw = pd.merge(closuresRaw,closuresRawActivity, on = ['Original Closure Number','Closure Number'], how = 'left')

#######################################


# sheet  = gc.open_by_url('https://docs.google.com/spreadsheets/d/1p9VJ0ATPxz68WMfR4uY0ZFb7YDN3Q51DVG69L2z88Go/edit?resourcekey#gid=1935498822')

# sheet =  sheet.get_worksheet(0)
# HORaw = pd.DataFrame(sheet.get_all_records())

spreadSheetId="1p9VJ0ATPxz68WMfR4uY0ZFb7YDN3Q51DVG69L2z88Go"
rangeName="Form Responses 1"
HORaw=get_data_df(spreadSheetId, rangeName)

HORaw.loc[(HORaw['VMS Relaxed HO?'] == "Yes") & (HORaw['VMS Handover Status (Relaxed HO)'] == 'Positive'), 'VMS Handover Status'] = 'Positive Relaxed'

HORaw = HORaw[['Timestamp', 'Email Address', 'Specify the HANDOVER MEETING ID',
       'Handover meeting date','City','Actual Society Name', 'Closure Product', 'Closure Type', 'Original Closure Number',
       'Total Occupied Flats', 'Total Sold flats ',
       'VMS Uniview Closure Number', 'VMS Handover Status',
       'ERP Uniview Closure Number', 'ERP Handover Status','Type of Handover',
       'Obstruction in Handover/Onboarding related - VMS','Obstruction in Handover/Onboarding related - ERP','SKU ID','VMS HO Done by','ERP HO Done by']]

HORaw[['Timestamp']] = HORaw[['Timestamp']].apply(changeDateTimeFormat)
HORaw[['Handover meeting date']] = HORaw[['Handover meeting date']].apply(changeDateFormat)
# HORaw['VMS Uniview Closure Number'] = pd.to_numeric(HORaw['VMS Uniview Closure Number'])
# HORaw['ERP Uniview Closure Number'] = pd.to_numeric(HORaw['ERP Uniview Closure Number'])


HOVMS = HORaw[['Timestamp', 'Email Address', 'Specify the HANDOVER MEETING ID',
       'Handover meeting date', 'Total Occupied Flats', 'Total Sold flats ',
       'VMS Uniview Closure Number', 'VMS Handover Status','Type of Handover','Obstruction in Handover/Onboarding related - VMS','SKU ID','VMS HO Done by']]

HOVMS = HOVMS[~(HOVMS['VMS Uniview Closure Number'].isna())]
HOVMS = HOVMS.sort_values(['Handover meeting date'], ascending = True).groupby(['VMS Uniview Closure Number']).head(1)

HOVMS.rename({'VMS Uniview Closure Number':'Closure Number','VMS Handover Status':'VMSHOStatus','Handover meeting date':'VMSHODate',
              'Total Sold flats ':'Sold Flats (VMS)', 'Total Occupied Flats' :'Occupied Flats (VMS)','Type of Handover':'Handover Type','Obstruction in Handover/Onboarding related - VMS':'Obstruction in VMS HO'},inplace=True,axis=1)
HOVMS['Sold Flats (VMS)'] = pd.to_numeric(HOVMS['Sold Flats (VMS)'], errors = 'coerce')

HOERP = HORaw[['Timestamp', 'Email Address', 'Specify the HANDOVER MEETING ID',
       'Handover meeting date', 'Total Occupied Flats', 'Total Sold flats ',
       'ERP Uniview Closure Number','ERP Handover Status','Type of Handover','Obstruction in Handover/Onboarding related - ERP','SKU ID','ERP HO Done by']]
HOERP = HOERP[~(HOERP['ERP Uniview Closure Number'].isna())]
HOERP = HOERP.sort_values(['Handover meeting date'], ascending = True).groupby(['ERP Uniview Closure Number']).head(1)

HOERP.rename({'ERP Uniview Closure Number':'Closure Number','ERP Handover Status':'ERPHOStatus','Handover meeting date':'ERPHODate',
              'Total Sold flats ':'Sold Flats (ERP)', 'Total Occupied Flats' :'Occupied Flats (ERP)','Type of Handover':'Handover Type','Obstruction in Handover/Onboarding related - ERP':'Obstruction in ERP HO'},inplace=True,axis=1)
HOERP['Sold Flats (ERP)'] = pd.to_numeric(HOERP['Sold Flats (ERP)'], errors = 'coerce')

closuresRaw = reduce(lambda left,right: pd.merge(left,right, on=['Closure Number'], how='left'),[closuresRaw,HOVMS,HOERP])
closuresRaw.rename({'Timestamp_x':'VMSHOFilledDate', 'Email Address_x':'VMSHOFilledEmail', 'Specify the HANDOVER MEETING ID_x':'VMSHOMeetingID','Handover Type_x':'VMS Handover Type','SKU ID_x':'VMS SKU ID',
                   'Timestamp_y':'ERPHOFilledDate', 'Email Address_y':'ERPHOFilledEmail', 'Specify the HANDOVER MEETING ID_y':'ERPHOMeetingID','Handover Type_y':'ERP Handover Type','SKU ID_y':'ERP SKU ID'}, inplace = True, axis = 1)
#closuresRaw[['VMSHOFilledDate','ERPHOFilledDate']] = closuresRaw[['VMSHOFilledDate','ERPHOFilledDate']].apply(changeDateFormat)

closuresRaw.loc[(closuresRaw['Sales Product'] == 'ERP only') & (closuresRaw['VMSHOStatus'].isna()) , 'VMSHOStatus'] = "Not Sold"
closuresRaw.loc[(closuresRaw['Sales Product'] == 'VMS only') & (closuresRaw['ERPHOStatus'].isna()) , 'ERPHOStatus'] = "Not Sold"


closuresRaw.loc[(closuresRaw['Sales Product'].isin({'VMS only','Both VMS + ERP'})) &  (closuresRaw['VMSHOStatus'].isna()) , 'VMSHOStatus'] = "Not Filled"
closuresRaw.loc[(closuresRaw['Sales Product'].isin({'ERP only','Both VMS + ERP'})) &  (closuresRaw['ERPHOStatus'].isna()) , 'ERPHOStatus'] = "Not Filled"

#closuresRaw['HO Product']=''

closuresRaw.loc[(closuresRaw['VMSHOStatus'].isin({"Positive","Positive Relaxed"})) & (closuresRaw['ERPHOStatus'] != "Positive"),'HO Product'] = 'VMS only'
closuresRaw.loc[(~closuresRaw['VMSHOStatus'].isin({"Positive","Positive Relaxed"})) & (closuresRaw['ERPHOStatus'] == "Positive"),'HO Product'] = 'ERP only'
closuresRaw.loc[(closuresRaw['VMSHOStatus'].isin({"Positive","Positive Relaxed"})) & (closuresRaw['ERPHOStatus'] == "Positive"),'HO Product'] = 'Both VMS + ERP'

###############################################################
closuresRaw['HO Product'].replace('nan',np.nan,inplace=True)

closuresRaw.loc[(~closuresRaw['Sold Flats (VMS)'].isna()) & (~closuresRaw['VMS Sold Flats'].isna()),'VMS Sold Flats Mismatch'] = closuresRaw['VMS Sold Flats'] - closuresRaw['Sold Flats (VMS)']
closuresRaw.loc[(~closuresRaw['Sold Flats (ERP)'].isna()) & (~closuresRaw['Sold Flats'].isna()),'ERP Sold Flats Mismatch'] = closuresRaw['Sold Flats'] - closuresRaw['Sold Flats (ERP)']

#########################
closuresRaw.loc[((closuresRaw['HO Product'] == "Both VMS + ERP") & (closuresRaw['Sales Product'] != "Both VMS + ERP")), 'WrongHOProduct'] = True

#########################

#closuresRaw.loc[:,'Closure Form Filling TAT'] = (closuresRaw['Closure Form Filled Date'] - closuresRaw['Closure Date']).apply(lambda x: x.days)


closuresRaw.loc[(~closuresRaw['VMSHOFilledDate'].isna()), 'VMSHOClosureFilledTAT'] = (closuresRaw['VMSHOFilledDate'] - pd.to_datetime(closuresRaw['Closure Form Filled Date'])).dt.days
closuresRaw.loc[(~closuresRaw['ERPHOFilledDate'].isna()), 'ERPHOClosureFilledTAT'] = (closuresRaw['ERPHOFilledDate'] - pd.to_datetime(closuresRaw['Closure Form Filled Date'])).dt.days

closuresRaw.loc[(~closuresRaw['VMSHODate'].isna()), 'VMSHOClosureTAT'] = (pd.to_datetime(closuresRaw['VMSHODate']) - pd.to_datetime(closuresRaw['Closure Date'])).dt.days
closuresRaw.loc[(~closuresRaw['ERPHODate'].isna()), 'ERPHOClosureTAT'] = (pd.to_datetime(closuresRaw['ERPHODate']) - pd.to_datetime(closuresRaw['Closure Date'])).dt.days

def teamtype(row):
    team = ['deepanshu.b@nobroker.in','aadit.vijay@nobroker.in','pravendra.verma@nobroker.in','naveen.pv@nobroker.in','deeksha.jaiswal@nobroker.in','manoj.d@nobroker.in','biswajit.pattasani@nobroker.in',
            'sonu.pradhan@nobroker.in','rohit.yadav1@nobroker.in','aniees.kajahussain@nobroker.in','abhilash.mishra@nobroker.in','avinash.mishra@nobroker.in','sayani.das@nobroker.in',
            'amar.tyagi@nobroker.in']
    if row['Closure Form Filled By'] in team:
      return 'ERP Sales Team'
    if row['Closure Form Filled By'] in ['shubham.shawarn@nobroker.in','anshuman.mishra@nobroker.in','arnab.rudra@nobroker.in','pritam.mazumdar@nobroker.in','debabrata.pandey@nobroker.in',
                                         'karthick.raja@nobroker.in', 'mugdha.bhutya@nobroker.in','divya.s@nobroker.in','pulkit.jain@nobroker.in','akshay.chandran@nobroker.in',
                                         'sandra.soman2@nobroker.in','sumit.kumar22@nobroker.in','chinthanraj.naik@nobroker.in','anshu.kumari@nobroker.in','prashant.dhalia6@nobroker.in','gajendra.choudhary@nobroker.in','tanveer.singh@nobroker.in']:
      return 'Sales-Central Team'
    else:
      return 'Sales Team'

closuresRaw['Team Category'] = closuresRaw.apply(teamtype, axis=1)

closuresRaw.loc[(closuresRaw['Closure Form Filled By'].isin(['deepanshu.b@nobroker.in'])) & (closuresRaw['Team Type'] == 'Sales team'), 'Team Category'] = 'Sales Team'

closuresRaw.loc[(closuresRaw['Closure Form Filled By'].isin(['anshuman.mishra@nobroker.in','sejal.soni@nobroker.in'])) & (closuresRaw['Team Type'] == 'ERP Sales team'), 'Team Category'] = 'ERP Sales Team'

closuresRaw.loc[(closuresRaw['Team Type'].isin(['Hybrid Sales'])), 'Team Category'] = 'Hybrid Team'


closuresRaw["Last Updated Datetime"] = pd.to_datetime(datetime.now()).tz_localize('UTC').tz_convert('Asia/Kolkata').strftime("%d-%B-%Y - %H:%M")

closuresRaw["Closure Status"] = "Valid"

columns_to_fill = ['New Devices', 'Subscription Revenue', 'Upfront Revenue (One time)', 'Access control/Boom Barrier net revenue', 'Revenue to be claimed']
closuresRaw[columns_to_fill] = closuresRaw[columns_to_fill].replace('',0)


closuresRaw = closuresRaw[['Closure Type', 'Closure Date', 'Original Closure Number','Closure Number', 'Closure Form Filled Date','Closure Form Filled Time', 'Closure Form Filled By',
        'Team Type','Lead Type', 'Closure Done By (Primary Email)','Primary BD City','Primary BD Role','Closure Done By (Secondary Email)','Secondary BD City', 'Secondary BD Role',
        'Society Type', 'Property Type','VMSTrialPeriod','ERPTrialPeriod', 'Society Name (Sales)','Closure City', 'Sales Product', 'HO Product', 'Total Flats', 'VMS Sold Flats',
        'ERP Sold Flats','Sold Flats', 'Occupancy Percentage', 'Pin Code','Society Latitude', 'Society Longitude', 'Rent of the society','New Devices', 'Refurbished Devices','Brand New Devices','Subscription Revenue',
        'Upfront Revenue (One time)', 'Access control/Boom Barrier net revenue','Welfare Amount', 'Monetization Advance Amount','RP (Revenue Points) to be claimed', 'Revenue to be claimed',
        'WrongPrimaryBDMail',	'WrongSecondaryBDMail',	'WrongHOProduct',	'CrossCityClosure',	'DuplicateClosureNumber',	'WrongOriginalClosureNumber',	'ClosureActivity',
        'VMSHOFilledDate', 'VMSHOFilledEmail', 'VMSHOMeetingID', 'VMSHODate','Occupied Flats (VMS)', 'Sold Flats (VMS)', 'VMSHOStatus','VMS Handover Type', 'Obstruction in VMS HO', 'VMS SKU ID','VMS HO Done by',
        'ERPHOFilledDate', 'ERPHOFilledEmail','ERPHOMeetingID', 'ERPHODate', 'Occupied Flats (ERP)','Sold Flats (ERP)', 'ERPHOStatus', 'ERP Handover Type','Obstruction in ERP HO', 'ERP SKU ID', 'ERP HO Done by',
        'VMS Sold Flats Mismatch', 'VMSHOClosureFilledTAT', 'VMSHOClosureTAT', 'ERP Sold Flats Mismatch','ERPHOClosureFilledTAT','ERPHOClosureTAT','Closure Form Filling TAT', 'HO Meeting ID',
        'Current VMS software used', 'Current ERP software used','Competition Proof Link (VMS)','Competition Proof Link (ERP)','Closure Proof Link','Agreement Type','Closure Status','Team Category',
        'Promised Mntz Revenue per annum to society','VMS Paid Subscription Tenure','ERP Paid Subscription Tenure','Biometrics Devices','Hood Rev Code','Last Updated Datetime']]

<ipython-input-99-e7325a0a2fbc>:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  closuresRaw.loc[closuresRaw['CrossCityClosure'] == False,'CrossCityClosure'] = ""
<ipython-input-99-e7325a0a2fbc>:174: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  closuresRaw['HO Product'].replace('nan',np.nan,inplace=True)


In [ ]:
spreadSheetId = "1BBIaBIbkKuQfVYt8W8VwbRPiztg29HPuBHzW764rEwE"
wb = gc.open_by_key(spreadSheetId)

ws = wb.worksheet("closureMasterRaw")
ws.clear()

# APPEND DATA TO SHEET
set_with_dataframe(ws, closuresRaw)

ws = wb.worksheet("HOMasterRaw")
ws.clear()

# APPEND DATA TO SHEET
set_with_dataframe(ws, HORaw[HORaw['Timestamp'] >= pd.to_datetime('01 September 2023')])

#**Read Corrected Data**

In [ ]:
sheet  = gc.open_by_url('https://docs.google.com/spreadsheets/d/1BBIaBIbkKuQfVYt8W8VwbRPiztg29HPuBHzW764rEwE/edit#gid=1501539375')
sheet =  sheet.get_worksheet(2)
closures = pd.DataFrame(sheet.get_all_records())

#to compare with static
closures2 = closures.copy()
closures2[['Closure Form Filled Time','VMSHOFilledDate','ERPHOFilledDate']] = closures2[['Closure Form Filled Time','VMSHOFilledDate','ERPHOFilledDate']].apply(changeDateTimeFormat)
closures2[['Closure Date','Closure Form Filled Date','VMSHODate','ERPHODate',]] = closures2[['Closure Date','Closure Form Filled Date',
                              'VMSHODate','ERPHODate']].apply(changeDateFormat)
closures2['Sold Flats'] = pd.to_numeric(closures2['Sold Flats'], errors = 'coerce')
closures2['VMS Sold Flats'] = pd.to_numeric(closures2['VMS Sold Flats'], errors = 'coerce')
closures2['ERP Sold Flats'] = pd.to_numeric(closures2['ERP Sold Flats'], errors = 'coerce')
closures2 = closures2.replace("TRUE",True)

###########################

closures[['Closure Form Filled Time']] = closures[['Closure Form Filled Time']].apply(changeDateTimeFormat)
deltaClosures = closuresRaw[closuresRaw['Closure Form Filled Time'] > closures['Closure Form Filled Time'].max()]
closures = pd.concat([closures, deltaClosures], ignore_index=True)

closures = pd.merge(closures, HR_Sales[['Emp Email ID','Emp City','Role']], left_on = 'Closure Done By (Primary Email)', right_on = 'Emp Email ID', how = 'left')
closures.drop({'ClosureActivity','DuplicateClosureNumber','WrongHOProduct','WrongOriginalClosureNumber','Primary BD City','Primary BD Role', 'Secondary BD City', 'Secondary BD Role'}, inplace = True, axis = 1)
closures.rename({'Emp City':'Primary BD City','Role':'Primary BD Role'}, inplace = True, axis = 1)
closures.drop({'Emp Email ID'}, inplace = True, axis = 1)

#closures['Closure Number'] = pd.to_numeric(closures['Closure Number'], errors = 'coerce')
closures['Sold Flats'] = pd.to_numeric(closures['Sold Flats'], errors = 'coerce')
closures['VMS Sold Flats'] = pd.to_numeric(closures['VMS Sold Flats'], errors = 'coerce')
closures['ERP Sold Flats'] = pd.to_numeric(closures['ERP Sold Flats'], errors = 'coerce')
closures.loc[closures['Sales Product'] == 'ERP only', 'VMS Sold Flats'] = 0
closures.loc[closures['Sales Product'] == 'VMS only', 'ERP Sold Flats'] = 0
closures['Sold Flats'] = closures[['ERP Sold Flats','VMS Sold Flats']].max(axis=1)

closures.loc[closures['Sales Product'] == 'ERP only', 'VMSTrialPeriod'] = ''
closures.loc[closures['Sales Product'] == 'VMS only', 'ERPTrialPeriod'] = ''

###########################


closures.loc[closures['Primary BD City'].isna(),'WrongPrimaryBDMail'] = True

closures = pd.merge(closures, HR_Sales[['Emp Email ID','Emp City','Role']], left_on = 'Closure Done By (Secondary Email)', right_on = 'Emp Email ID', how = 'left')
closures.rename({'Emp City':'Secondary BD City','Role':'Secondary BD Role'}, inplace = True, axis = 1)
closures.drop({'Emp Email ID'}, inplace = True, axis = 1)

closures.loc[(closures['Secondary BD City'].isna()) & (closures['Closure Done By (Secondary Email)'] != ""),'WrongSecondaryBDMail'] = True

###########################

#closures = closures.sort_values(['Closure Date'], ascending = True).groupby(['Original Closure Number','Closure Number']).head(5)

closuresDuplicateClosureNumber = closures[closures['Closure Number'].duplicated()]['Closure Number'].drop_duplicates()
closures.loc[closures['Closure Number'].isin(closuresDuplicateClosureNumber),'DuplicateClosureNumber'] = True

##########################

# Closure Date Remapping(Max of 3 days from Closure Form filled date)
closures[['Closure Date','Closure Form Filled Date','VMSHODate','ERPHODate']] = closures[['Closure Date','Closure Form Filled Date'
                              ,'VMSHODate','ERPHODate']].apply(changeDateFormat)


closures['Closure Date'] = closures.apply(lambda x: max(pd.Timestamp(x['Closure Date']),
                              (pd.Timestamp(x['Closure Form Filled Date']) - relativedelta(days=3)))
                              if pd.Timestamp(x['Closure Date']) >= pd.to_datetime('2023-10-01') else x['Closure Date'], axis=1)

closures['Closure Date'] = pd.to_datetime(closures['Closure Date']).dt.date

###########################
closures.loc[:,'Closure Form Filling TAT'] = (closures['Closure Form Filled Date'] - closures['Closure Date']).apply(lambda x: x.days)

###########################

closures.loc[:,'CrossCityClosure'] = (closures['Closure City'] != closures['Primary BD City']) | ( (~closures['Secondary BD City'].isna()) & (closures['Closure City'] != closures['Secondary BD City']))
closures.loc[closures['CrossCityClosure'] == False,'CrossCityClosure'] = ""

###########################

closures.loc[(closures['Closure Type'].isin({'Upsell (VMS/ERP)','Winback'})) & (closures['Closure Number'] == closures['Original Closure Number']),'WrongOriginalClosureNumber'] = True

###########################

closuresActivity = closures[closures['Closure Type'].isin({'Upsell (VMS/ERP)','Winback'})]

closuresActivity = closures.sort_values(['Closure Date'], ascending = True).groupby(['Original Closure Number']).head(2)
closuresActivity = closuresActivity[(closuresActivity['Original Closure Number'].duplicated(keep= False)) & (closuresActivity['Original Closure Number']!= "")]

closuresActivity.loc[:, 'Closure Month'] = pd.to_datetime(closuresActivity['Closure Date']).dt.month
closuresActivity['Months Since Last Closure'] = closuresActivity.groupby(['Original Closure Number'])['Closure Month'].transform(lambda x: x.diff())
closuresActivity['Days Since Last Closure'] = closuresActivity.groupby(['Original Closure Number'])['Closure Date'].transform(lambda x: x.diff().dt.days)

closuresActivity.sort_index(inplace=True)

closuresActivity.loc[closuresActivity['Days Since Last Closure']<=90, 'ClosureActivity'] = 'Within 90 days'
closuresActivity.loc[closuresActivity['Days Since Last Closure']<=35, 'ClosureActivity'] = 'Within 35 days'
closuresActivity.loc[closuresActivity['Months Since Last Closure']==0, 'ClosureActivity'] = 'Within Month Closure Activity'


#closuresActivity = closuresActivity[~closuresActivity['ClosureActivity'].isna()][['Original Closure Number','Closure Number','ClosureActivity']].drop_duplicates() Old Code Not able to read 'nan'

closuresActivity = closuresActivity[(closuresActivity['ClosureActivity'].notna()) & (closuresActivity['ClosureActivity']!='nan')][['Original Closure Number','Closure Number','ClosureActivity']].drop_duplicates()


closures = pd.merge(closures,closuresActivity, on = ['Original Closure Number','Closure Number'], how = 'left')

closures[columns_to_fill] = closures[columns_to_fill].replace('',0)

closures = closures[['Closure Type','Closure Date','Original Closure Number','Closure Number','Closure Form Filled Date', 'Closure Form Filled Time',
                      'Closure Form Filled By','Team Type', 'Lead Type', 'Closure Done By (Primary Email)','Closure Done By (Secondary Email)',
                      'Primary BD City', 'Primary BD Role','Secondary BD City', 'Secondary BD Role',
                      'Society Type', 'Property Type','VMSTrialPeriod','ERPTrialPeriod','Society Name (Sales)','Closure City','Sales Product',
                      'Total Flats','Sold Flats','VMS Sold Flats','ERP Sold Flats','Occupancy Percentage','Pin Code','Society Latitude', 'Society Longitude',
                      'Rent of the society','New Devices','Refurbished Devices','Subscription Revenue','Upfront Revenue (One time)',
                      'Access control/Boom Barrier net revenue', 'Welfare Amount', 'Monetization Advance Amount', 'RP (Revenue Points) to be claimed', 'Revenue to be claimed',
                      'WrongPrimaryBDMail','WrongSecondaryBDMail','CrossCityClosure','DuplicateClosureNumber','WrongOriginalClosureNumber','ClosureActivity',
                      'Closure Form Filling TAT', 'HO Meeting ID','Current VMS software used','Current ERP software used','Competition Proof Link (VMS)','Competition Proof Link (ERP)',
                      'Closure Proof Link','Agreement Type','Closure Status','Team Category','Last Updated Datetime','Brand New Devices','Biometrics Devices','Promised Mntz Revenue per annum to society',
                      'VMS Paid Subscription Tenure','ERP Paid Subscription Tenure','Hood Rev Code']]

###########################

# spreadsheetid = "11zTVPIvJjNS2g2l8-m3wn5yyqtC0Knhm_bVyQ2vRP4U"
# range = "HOMasterCorrected"
# HO = get_data_df(spreadsheetid,range)

sheet  = gc.open_by_url('https://docs.google.com/spreadsheets/d/1BBIaBIbkKuQfVYt8W8VwbRPiztg29HPuBHzW764rEwE/edit?gid=289575185#gid=289575185')
sheet =  sheet.get_worksheet(3)
HO = pd.DataFrame(sheet.get_all_records())

HO[['Timestamp']] = HO[['Timestamp']].apply(changeDateTimeFormat)
deltaHOs = HORaw[HORaw['Timestamp'] > HO['Timestamp'].max()]
HO = pd.concat([HO,deltaHOs], ignore_index = True)
#closures = pd.concat([closures, deltaClosures], ignore_index=True)

HO = HO[['Timestamp', 'Email Address', 'Specify the HANDOVER MEETING ID',
       'Handover meeting date','City','Actual Society Name', 'Closure Product', 'Closure Type', 'Original Closure Number',
       'Total Occupied Flats', 'Total Sold flats ','VMS Uniview Closure Number', 'VMS Handover Status',
       'ERP Uniview Closure Number', 'ERP Handover Status','Type of Handover','Obstruction in Handover/Onboarding related - VMS','Obstruction in Handover/Onboarding related - ERP','SKU ID','VMS HO Done by','ERP HO Done by']]

HO[['Handover meeting date']] = HO[['Handover meeting date']].apply(changeDateFormat)
# HO['VMS Uniview Closure Number'] = pd.to_numeric(HO['VMS Uniview Closure Number'])
# HO['ERP Uniview Closure Number'] = pd.to_numeric(HO['ERP Uniview Closure Number'])

HOVMS = HO[['Timestamp', 'Email Address', 'Specify the HANDOVER MEETING ID',
       'Handover meeting date', 'Total Occupied Flats', 'Total Sold flats ',
       'VMS Uniview Closure Number', 'VMS Handover Status','Type of Handover','Obstruction in Handover/Onboarding related - VMS','SKU ID','VMS HO Done by']]

HOVMS = HOVMS[(HOVMS['VMS Uniview Closure Number']!='')]
HOVMS = HOVMS.sort_values(['Handover meeting date'], ascending = True).groupby(['VMS Uniview Closure Number']).head(1)

HOVMS.rename({'VMS Uniview Closure Number':'Closure Number','VMS Handover Status':'VMSHOStatus','Handover meeting date':'VMSHODate',
              'Total Sold flats ':'Sold Flats (VMS)', 'Total Occupied Flats' :'Occupied Flats (VMS)','Type of Handover':'Handover Type','Obstruction in Handover/Onboarding related - VMS':'Obstruction in VMS HO'},inplace=True,axis=1)
HOVMS['Sold Flats (VMS)'] = pd.to_numeric(HOVMS['Sold Flats (VMS)'], errors = 'coerce')

HOERP = HO[['Timestamp', 'Email Address', 'Specify the HANDOVER MEETING ID',
       'Handover meeting date', 'Total Occupied Flats', 'Total Sold flats ',
       'ERP Uniview Closure Number','ERP Handover Status','Type of Handover','Obstruction in Handover/Onboarding related - ERP','SKU ID','ERP HO Done by']]


#########################################################
HOERP = HOERP[HOERP['ERP Uniview Closure Number']!='']
HOERP = HOERP.sort_values(['Handover meeting date'], ascending = True).groupby(['ERP Uniview Closure Number']).head(1)

HOERP.rename({'ERP Uniview Closure Number':'Closure Number','ERP Handover Status':'ERPHOStatus','Handover meeting date':'ERPHODate',
              'Total Sold flats ':'Sold Flats (ERP)', 'Total Occupied Flats' :'Occupied Flats (ERP)','Type of Handover':'Handover Type','Obstruction in Handover/Onboarding related - ERP':'Obstruction in ERP HO'},inplace=True,axis=1)
HOERP['Sold Flats (ERP)'] = pd.to_numeric(HOERP['Sold Flats (ERP)'], errors = 'coerce')

closures = reduce(lambda left,right: pd.merge(left,right, on=['Closure Number'], how='left'),[closures,HOVMS,HOERP])
closures.rename({'Timestamp_x':'VMSHOFilledDate', 'Email Address_x':'VMSHOFilledEmail', 'Specify the HANDOVER MEETING ID_x':'VMSHOMeetingID','Handover Type_x':'VMS Handover Type','SKU ID_x':'VMS SKU ID',
                   'Timestamp_y':'ERPHOFilledDate', 'Email Address_y':'ERPHOFilledEmail', 'Specify the HANDOVER MEETING ID_y':'ERPHOMeetingID','Handover Type_y':'ERP Handover Type','SKU ID_y':'ERP SKU ID'}, inplace = True, axis = 1)
#closures[['VMSHOFilledDate','ERPHOFilledDate']] = closures[['VMSHOFilledDate','ERPHOFilledDate']].apply(changeDateFormat)

closures.loc[(closures['Sales Product'] == 'ERP only') & (closures['VMSHOStatus'].isna()) , 'VMSHOStatus'] = "Not Sold"
closures.loc[(closures['Sales Product'] == 'VMS only') & (closures['ERPHOStatus'].isna()) , 'ERPHOStatus'] = "Not Sold"

closures.loc[(closures['Sales Product'].isin({'VMS only','Both VMS + ERP'})) &  (closures['VMSHOStatus'].isna()) , 'VMSHOStatus'] = "Not Filled"
closures.loc[(closures['Sales Product'].isin({'ERP only','Both VMS + ERP'})) &  (closures['ERPHOStatus'].isna()) , 'ERPHOStatus'] = "Not Filled"


closures.loc[(closures['VMSHOStatus'].isin({"Positive","Positive Relaxed"})) & (closures['ERPHOStatus'] != "Positive"),'HO Product'] = 'VMS only'
closures.loc[(~closures['VMSHOStatus'].isin({"Positive","Positive Relaxed"})) & (closures['ERPHOStatus'] == "Positive"),'HO Product'] = 'ERP only'
closures.loc[(closures['VMSHOStatus'].isin({"Positive","Positive Relaxed"})) & (closures['ERPHOStatus'] == "Positive"),'HO Product'] = 'Both VMS + ERP'

#############################################################
closures['HO Product'].replace('nan',np.nan,inplace=True)

closures.loc[(~closures['Sold Flats (VMS)'].isna()) & (~closures['VMS Sold Flats'].isna()),'VMS Sold Flats Mismatch'] = closures['VMS Sold Flats'] - closures['Sold Flats (VMS)']
closures.loc[(~closures['Sold Flats (ERP)'].isna()) & (~closures['Sold Flats'].isna()),'ERP Sold Flats Mismatch'] = closures['Sold Flats'] - closures['Sold Flats (ERP)']

#########################
closures.loc[((closures['HO Product'] == "Both VMS + ERP") & (closures['Sales Product'] != "Both VMS + ERP")), 'WrongHOProduct'] = True


#########################

closures.loc[(~closures['VMSHOFilledDate'].isna()), 'VMSHOClosureFilledTAT'] = (closures['VMSHOFilledDate'] - pd.to_datetime(closures['Closure Form Filled Date'])).dt.days
closures.loc[(~closures['ERPHOFilledDate'].isna()), 'ERPHOClosureFilledTAT'] = (closures['ERPHOFilledDate'] - pd.to_datetime(closures['Closure Form Filled Date'])).dt.days

closures.loc[(~closures['VMSHODate'].isna()), 'VMSHOClosureTAT'] = (pd.to_datetime(closures['VMSHODate']) - pd.to_datetime(closures['Closure Date'])).dt.days
closures.loc[(~closures['ERPHODate'].isna()), 'ERPHOClosureTAT'] = (pd.to_datetime(closures['ERPHODate']) - pd.to_datetime(closures['Closure Date'])).dt.days

closures.loc[:,'Closure Month'] = pd.to_datetime(closures['Closure Date']).dt.month

closures.loc[closures['Closure City'].isin({'Kolkata','Nagpur','Jaipur','Kochi'}), 'City Type'] = 'Emerging City'
closures.loc[~closures['Closure City'].isin({'Kolkata','Nagpur','Jaipur','Kochi'}), 'City Type'] = 'Main City'
closures.loc[closures['Closure City'] == 'Other City (Non-operational Cities)', 'City Type'] = 'Other City'

closures['monthDays']=pd.to_datetime(closures['Closure Date']).dt.day
closures['year']=pd.to_datetime(closures['Closure Date']).dt.year
closures['month']=pd.to_datetime(closures['Closure Date']).dt.month

closures['Occupied Flats (VMS)'] = closures['Occupied Flats (VMS)'].fillna(0)
closures['Occupied Flats (ERP)'] = closures['Occupied Flats (ERP)'].fillna(0)


def teamtype(row):
   team = ['deepanshu.b@nobroker.in','aadit.vijay@nobroker.in','pravendra.verma@nobroker.in','naveen.pv@nobroker.in','deeksha.jaiswal@nobroker.in','manoj.d@nobroker.in','biswajit.pattasani@nobroker.in',
           'sonu.pradhan@nobroker.in','rohit.yadav1@nobroker.in','aniees.kajahussain@nobroker.in','abhilash.mishra@nobroker.in','avinash.mishra@nobroker.in','sayani.das@nobroker.in',
           'goundla.karthik@nobroker.in','murali.v1@nobroker.in','prashant.dhalia@nobroker.in','vivechan.ka@nobroker.in','amar.tyagi@nobroker.in']
   if row['Closure Form Filled By'] in team:
     return 'FSD Team'
   if row['Closure Form Filled By'] in ['shubham.shawarn@nobroker.in','anshuman.mishra@nobroker.in','arnab.rudra@nobroker.in','pritam.mazumdar@nobroker.in','debabrata.pandey@nobroker.in',
                                        'karthick.raja@nobroker.in','mugdha.bhutya@nobroker.in','divya.s@nobroker.in','sejal.soni@nobroker.in','pulkit.jain@nobroker.in','akshay.chandran@nobroker.in',
                                        'sandra.soman2@nobroker.in','sumit.kumar22@nobroker.in','chinthanraj.naik@nobroker.in','anshu.kumari@nobroker.in','prashant.dhalia6@nobroker.in','gajendra.choudhary@nobroker.in','tanveer.singh@nobroker.in']:
     return 'Sales-Central Team'
   else:
     return 'Sales Team'

closures['Team Category'] = closures.apply(teamtype, axis=1)

closures.loc[(closures['Closure Form Filled By'].isin(['deepanshu.b@nobroker.in'])) & (closures['Team Type'] == 'Sales team'), 'Team Category'] = 'Sales Team'
closures.loc[(closures['Closure Form Filled By'].isin(['anshuman.mishra@nobroker.in','sejal.soni@nobroker.in'])) & (closures['Team Type'] == 'ERP Sales team'), 'Team Category'] = 'FSD Team'
closures.loc[(closures['Team Type'].isin(['Hybrid Sales'])), 'Team Category'] = 'Hybrid Team'

closures.loc[closures["Last Updated Datetime"].isna(),"Last Updated Datetime"] = pd.to_datetime(datetime.now()).tz_localize('UTC').tz_convert('Asia/Kolkata').strftime("%d-%b-%Y - %H:%M")

closures[["Last Updated Datetime"]] = closures[["Last Updated Datetime"]].apply(changeDateTimeFormat)

closures[['VMSHOFilledDate','ERPHOFilledDate']] = closures[['VMSHOFilledDate','ERPHOFilledDate']].apply(changeDateTimeFormat)
closures['Last Updated Datetime'] = closures[['Last Updated Datetime','VMSHOFilledDate','ERPHOFilledDate']].max(axis=1)
closures.loc[(closures['Team Type'].str.contains('Joint Closure')) & (closures['VMSHOStatus']=='Positive'),'HO Product'] = closures['Sales Product']
closures.loc[(closures['Team Type'].str.contains('Joint Closure')) & (closures['ERPHOStatus']=='Positive'),'HO Product'] = closures['Sales Product']
closures.loc[(closures['Team Type'].str.contains('Joint Closure')) & (closures['VMSHOStatus'].isin(['Not Filled','Negative']) & (closures['Sales Product']=='VMS only')),'HO Product'] = ''
closures.loc[(closures['Team Type'].str.contains('Joint Closure')) & (closures['ERPHOStatus'].isin(['Not Filled','Negative']) & (closures['Sales Product']=='ERP only')),'HO Product'] = ''
closures.loc[(closures['Team Type'].str.contains('Joint Closure')) & (closures['Sales Product']=='ERP only'),'VMSHOStatus'] = 'Not Sold'
closures.loc[(closures['Team Type'].str.contains('Joint Closure')) & (closures['Sales Product']=='VMS only'),'ERPHOStatus'] = 'Not Sold'

closures.loc[(closures['Closure Type'].str.contains('Addons')),'HO Product'] = closures['Sales Product']

closures['Occupancy Percentage'] = closures['Occupancy Percentage'].astype(int)

def Occupancy_Multiplier(row):
  if row['Occupancy Percentage']>=0 and row['Occupancy Percentage']<2 and pd.Timestamp(row['Closure Date'])<=pd.to_datetime("30 Nov 2023"):
    return row['VMS Sold Flats']*0.40
  elif row['Occupancy Percentage']>=2 and row['Occupancy Percentage']<5 and pd.Timestamp(row['Closure Date'])<=pd.to_datetime("30 Nov 2023"):
    return row['VMS Sold Flats']*0.65
  elif row['Occupancy Percentage']>=5 and row['Occupancy Percentage']<10 and pd.Timestamp(row['Closure Date'])<=pd.to_datetime("30 Nov 2023"):
    return row['VMS Sold Flats']*0.65
  elif row['Occupancy Percentage']>=10 and row['Occupancy Percentage']<50 and pd.Timestamp(row['Closure Date'])<=pd.to_datetime("30 Nov 2023"):
    return row['VMS Sold Flats']*0.75
  elif row['Occupancy Percentage']>=0 and row['Occupancy Percentage']<2 and pd.Timestamp(row['Closure Date'])>pd.to_datetime("30 Nov 2023") and pd.Timestamp(row['Closure Date'])<=pd.to_datetime("26 Jan 2024"):
    return row['VMS Sold Flats']*0.40
  elif row['Occupancy Percentage']>=2 and row['Occupancy Percentage']<5 and pd.Timestamp(row['Closure Date'])>pd.to_datetime("30 Nov 2023") and pd.Timestamp(row['Closure Date'])<=pd.to_datetime("26 Jan 2024"):
    return row['VMS Sold Flats']*0.50
  elif row['Occupancy Percentage']>=5 and row['Occupancy Percentage']<10 and pd.Timestamp(row['Closure Date'])>pd.to_datetime("30 Nov 2023") and pd.Timestamp(row['Closure Date'])<=pd.to_datetime("26 Jan 2024"):
    return row['VMS Sold Flats']*0.50
  elif row['Occupancy Percentage']>=10 and row['Occupancy Percentage']<50 and pd.Timestamp(row['Closure Date'])>pd.to_datetime("30 Nov 2023"):
    return row['VMS Sold Flats']*0.65
  elif row['Occupancy Percentage']>=0 and row['Occupancy Percentage']<2 and pd.Timestamp(row['Closure Date'])>pd.to_datetime("26 Jan 2024"):
    return row['VMS Sold Flats']*0.20
  elif row['Occupancy Percentage']>=2 and row['Occupancy Percentage']<5 and pd.Timestamp(row['Closure Date'])>pd.to_datetime("26 Jan 2024"):
    return row['VMS Sold Flats']*0.40
  elif row['Occupancy Percentage']>=5 and row['Occupancy Percentage']<10 and pd.Timestamp(row['Closure Date'])>pd.to_datetime("26 Jan 2024"):
    return row['VMS Sold Flats']*0.40
  else:
    return row['VMS Sold Flats']

closures['VMS Sold Flats as per Occupancy'] = closures.apply(Occupancy_Multiplier,axis = 1)
closures['VMS Sold Flats as per Occupancy'] = closures['VMS Sold Flats as per Occupancy'].round(0).astype(int)

closures = closures[['Closure Type', 'Closure Date', 'Original Closure Number','Closure Number', 'Closure Form Filled Date','Closure Form Filled Time','Closure Month', 'Closure Form Filled By',
       'Team Type','Lead Type', 'Closure Done By (Primary Email)','Primary BD City','Primary BD Role','Closure Done By (Secondary Email)','Secondary BD City', 'Secondary BD Role',
       'Society Type', 'Property Type','VMSTrialPeriod','ERPTrialPeriod', 'Society Name (Sales)','Closure City','City Type', 'Sales Product', 'HO Product', 'Total Flats', 'VMS Sold Flats',
       'ERP Sold Flats','Sold Flats','Occupancy Percentage','Pin Code','Society Latitude','Society Longitude','Rent of the society','New Devices','Refurbished Devices','Subscription Revenue',
       'Upfront Revenue (One time)', 'Access control/Boom Barrier net revenue','Welfare Amount', 'Monetization Advance Amount','RP (Revenue Points) to be claimed', 'Revenue to be claimed',
       'WrongPrimaryBDMail',	'WrongSecondaryBDMail',	'WrongHOProduct',	'CrossCityClosure',	'DuplicateClosureNumber',	'WrongOriginalClosureNumber',	'ClosureActivity',
       'VMSHOFilledDate', 'VMSHOFilledEmail', 'VMSHOMeetingID', 'VMSHODate','Occupied Flats (VMS)', 'Sold Flats (VMS)', 'VMSHOStatus','VMS Handover Type', 'Obstruction in VMS HO', 'VMS SKU ID','VMS HO Done by',
       'ERPHOFilledDate', 'ERPHOFilledEmail','ERPHOMeetingID', 'ERPHODate', 'Occupied Flats (ERP)','Sold Flats (ERP)', 'ERPHOStatus', 'ERP Handover Type','Obstruction in ERP HO', 'ERP SKU ID', 'ERP HO Done by',
       'VMS Sold Flats Mismatch', 'VMSHOClosureFilledTAT', 'VMSHOClosureTAT', 'ERP Sold Flats Mismatch','ERPHOClosureFilledTAT','ERPHOClosureTAT','Closure Form Filling TAT', 'monthDays','year','HO Meeting ID',
       'Current VMS software used','Current ERP software used','Competition Proof Link (VMS)','Competition Proof Link (ERP)','Closure Proof Link','Agreement Type','Closure Status','Team Category','Last Updated Datetime',
        'VMS Sold Flats as per Occupancy','Brand New Devices','Biometrics Devices','Promised Mntz Revenue per annum to society','VMS Paid Subscription Tenure','ERP Paid Subscription Tenure','Hood Rev Code']]

<ipython-input-101-a75c33eb40fa>:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  closures = pd.concat([closures, deltaClosures], ignore_index=True)
<ipython-input-101-a75c33eb40fa>:184: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  closures['HO Product'].replace('nan',np.nan,inpl

In [ ]:
closures3=closures.copy()
closures3['VMSHOFilledDate'] = closures3['VMSHOFilledDate'].apply(lambda x: str(x))
closures3['ERPHOFilledDate'] = closures3['ERPHOFilledDate'].apply(lambda x: str(x))

closures2['VMSHOFilledDate'] = closures2['VMSHOFilledDate'].apply(lambda x: str(x))
closures2['ERPHOFilledDate'] = closures2['ERPHOFilledDate'].apply(lambda x: str(x))


In [ ]:
closures2.fillna('').replace(pd.NaT,'').loc[10,'Primary BD Role'],closures3.fillna('').replace(pd.NaT,'').loc[10,'Primary BD Role']

<ipython-input-103-73e6c74cab99>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  closures2.fillna('').replace(pd.NaT,'').loc[10,'Primary BD Role'],closures3.fillna('').replace(pd.NaT,'').loc[10,'Primary BD Role']


('SR.TSM', 'SR.TSM')

In [ ]:
changed_rows = closures[closures2.fillna('').replace(pd.NaT,'').ne(closures3.fillna('').replace(pd.NaT,'')).any(axis=1)]
changed_rows

<ipython-input-104-347e8425feb7>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  changed_rows = closures[closures2.fillna('').replace(pd.NaT,'').ne(closures3.fillna('').replace(pd.NaT,'')).any(axis=1)]


,Closure Type,Closure Date,Original Closure Number,Closure Number,Closure Form Filled Date,Closure Form Filled Time,Closure Month,Closure Form Filled By,Team Type,Lead Type,...,Closure Status,Team Category,Last Updated Datetime,VMS Sold Flats as per Occupancy,Brand New Devices,Biometrics Devices,Promised Mntz Revenue per annum to society,VMS Paid Subscription Tenure,ERP Paid Subscription Tenure,Hood Rev Code
2911,New Closure,2024-09-08,1037843,1037843,2024-09-08,2024-09-08 23:09:26,9,satyakishore.y@nobroker.in,Sales team,Central,...,Valid,Sales Team,2025-03-03 13:05:19,117,1,0,0.0,0 year (No Payment Received),0 year (No Payment Received),
3125,New Closure,2024-09-24,1038078,1038078,2024-09-27,2024-09-27 16:14:20,9,debabrata.pandey@nobroker.in,Central Sales team,Central,...,Valid,Sales-Central Team,2025-03-03 13:05:24,0,0,0,0.0,VMS not sold - NA,2 Year (Limited period),
3128,New Closure,2024-09-28,1038083,1038083,2024-09-29,2024-09-29 10:36:23,9,satyakishore.y@nobroker.in,Sales team,City,...,Valid,Sales Team,2025-03-03 13:05:29,67,2,0,0.0,One time payment (Upfront Revenue),ERP not sold - NA,
3165,New Closure,2024-10-08,1038149,1038149,2024-10-11,2024-10-11 19:45:35,10,gajendra.choudhary@nobroker.in,Central Sales team,City,...,Valid,Sales-Central Team,2025-03-03 13:05:46,215,1,1,0.0,4 Year (Limited period),ERP not sold - NA,
3261,New Closure,2024-10-12,1038226,1038226,2024-10-15,2024-10-15 17:34:12,10,gajendra.choudhary@nobroker.in,Central Sales team,City,...,Valid,Sales-Central Team,2025-03-03 13:06:03,204,2,0,0.0,1 Year (Limited period),ERP not sold - NA,
3412,New Closure,2024-10-26,1038317,1038317,2024-10-27,2024-10-27 03:10:12,10,satyakishore.y@nobroker.in,Sales team,City,...,Valid,Sales Team,2025-03-03 13:06:09,60,1,0,0.0,One time payment (Upfront Revenue),One time payment (Upfront Revenue),
3416,New Closure,2024-10-30,1038377,1038377,2024-10-31,2024-10-31 15:22:58,10,satyakishore.y@nobroker.in,Sales team,City,...,Valid,Sales Team,2025-03-03 13:06:14,65,1,0,0.0,0 year (No Payment Received),ERP not sold - NA,
3590,New Closure,2024-11-23,1038536,1038536,2024-11-23,2024-11-23 20:09:09,11,satyakishore.y@nobroker.in,Sales team,City,...,Valid,Sales Team,2025-03-03 13:06:28,70,1,0,0.0,0 year (No Payment Received),ERP not sold - NA,
3741,New Closure,2024-12-10,1038713,1038713,2024-12-10,2024-12-10 23:34:43,12,satyakishore.y@nobroker.in,Sales team,City,...,Valid,Sales Team,2025-03-03 13:06:34,96,1,0,0.0,0 year (No Payment Received),0 year (No Payment Received),
4353,New Closure,2025-01-26,1039252,1039252,2025-01-26,2025-01-26 22:45:44,1,prasad.chavan@nobroker.in,Sales team,Central,...,Marked Dummy,Sales Team,2025-03-03 13:06:43,228,1,0,0.0,5 Year (Limited period),ERP not sold - NA,Hood-Rev-0000


In [ ]:
spreadSheetId = "1-sGZbZ21jBHAZt2Z4Pe_XRVp-DZkz89OALCgzkAwapY"
wb = gc.open_by_key(spreadSheetId)

ws = wb.worksheet("Sheet2")
set_with_dataframe(ws,closures2.fillna('').replace(pd.NaT,'').ne(closures3.fillna('').replace(pd.NaT,'')))

<ipython-input-105-f2a6e5f5738e>:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  set_with_dataframe(ws,closures2.fillna('').replace(pd.NaT,'').ne(closures3.fillna('').replace(pd.NaT,'')))


In [ ]:
closures[['Closure Month','Closure Form Filling TAT','Occupied Flats (VMS)','Occupied Flats (ERP)','monthDays','year']] = closures[['Closure Month','Closure Form Filling TAT','Occupied Flats (VMS)','Occupied Flats (ERP)','monthDays','year',]].astype(float)

In [ ]:
#To open the Google Sheets workbook and select the worksheet
spreadSheetId = "1BBIaBIbkKuQfVYt8W8VwbRPiztg29HPuBHzW764rEwE"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet("closureMasterCorrected")

# # APPEND DATA TO SHEET
# set_with_dataframe(ws, closures)

# Loop through rows and values in the DataFrame
for i, v in changed_rows.iterrows():
    df = pd.DataFrame(closures.iloc[i, :]).T
    df['Last Updated Datetime'] = pd.to_datetime(datetime.now()).tz_localize('UTC').tz_convert('Asia/Kolkata').strftime("%Y-%m-%d %H:%M:%S")
    df = df.replace(np.nan, None)
    df = df.replace(np.inf, None)

    # Convert the DataFrame to a dictionary
    df_dict = df.to_dict(orient='list')

    # Update the Google Sheet with the dictionary data
    set_with_dataframe(ws, pd.DataFrame.from_dict(df_dict), row=i+2, include_column_header=False)

# To Append Corrected HO data
ws = wb.worksheet("HOMasterCorrected")
ws.clear()

set_with_dataframe(ws, HO[HO['Timestamp'] >= pd.to_datetime('01 September 2023')])

#**Sales RP Calculation**

In [ ]:
closures.loc[closures['Closure Number'].isin([1035008,1035000,1035411,1035251,1035215,1038178]),'Primary BD City']=closures['Closure City']

In [ ]:
RP_D4_HO = closures.loc[closures['Closure Status']!="Marked Dummy"]

In [ ]:
RP_D4_HO = RP_D4_HO[['Closure Type', 'Closure Date', 'Original Closure Number','Closure Number', 'Closure Form Filled Date','Closure Form Filled Time','Closure Month', 'Closure Form Filled By',
       'Team Type','Lead Type', 'Closure Done By (Primary Email)','Primary BD City','Primary BD Role','Closure Done By (Secondary Email)','Secondary BD City', 'Secondary BD Role',
       'Society Type', 'Property Type','VMSTrialPeriod','ERPTrialPeriod', 'Society Name (Sales)','Closure City','City Type', 'Sales Product', 'HO Product', 'Total Flats', 'VMS Sold Flats',
       'ERP Sold Flats','Sold Flats','Occupancy Percentage','Pin Code','Society Latitude','Society Longitude','Rent of the society','New Devices','Refurbished Devices','Brand New Devices','Biometrics Devices',
       'Subscription Revenue', 'Upfront Revenue (One time)', 'Access control/Boom Barrier net revenue','Welfare Amount', 'Monetization Advance Amount','RP (Revenue Points) to be claimed', 'Revenue to be claimed',
       'WrongPrimaryBDMail',	'WrongSecondaryBDMail',	'WrongHOProduct',	'CrossCityClosure',	'DuplicateClosureNumber',	'WrongOriginalClosureNumber',	'ClosureActivity',
       'VMSHOFilledDate', 'VMSHOFilledEmail', 'VMSHOMeetingID', 'VMSHODate','Occupied Flats (VMS)', 'Sold Flats (VMS)', 'VMSHOStatus','VMS Handover Type', 'Obstruction in VMS HO', 'VMS SKU ID','VMS HO Done by',
       'ERPHOFilledDate', 'ERPHOFilledEmail','ERPHOMeetingID', 'ERPHODate', 'Occupied Flats (ERP)','Sold Flats (ERP)', 'ERPHOStatus', 'ERP Handover Type','Obstruction in ERP HO', 'ERP SKU ID', 'ERP HO Done by',
       'VMS Sold Flats Mismatch', 'VMSHOClosureFilledTAT', 'VMSHOClosureTAT', 'ERP Sold Flats Mismatch','ERPHOClosureFilledTAT','ERPHOClosureTAT','Closure Form Filling TAT', 'monthDays','year','HO Meeting ID',
       'Current VMS software used','Current ERP software used','Competition Proof Link (VMS)','Competition Proof Link (ERP)','Closure Proof Link','Agreement Type','Closure Status','Team Category',
       'Last Updated Datetime','VMS Sold Flats as per Occupancy']]

In [ ]:
spreadSheetId="1SqiNSq8YUm61Ai3OKPV4QpRBOTNztrVYG2AkO3jZwe4"
rangeName="RP_D4"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
#ws.clear()
wb.values_clear("RP_D4!A5:CN")
row = 5
col = 1
df = pd.DataFrame(RP_D4_HO)
set_with_dataframe(ws, df.iloc[:,:92],row = row,col=col)

In [ ]:
sheet  = gc.open_by_url('https://docs.google.com/spreadsheets/d/1DZPE9U2OZ9W5KwUIe0Vmj36yKHRSxT0CUSoKHvUxbIE/edit#gid=0')
sheet =  sheet.get_worksheet(0)
PIAmultiplier = pd.DataFrame(sheet.get_all_records())

PIAmultiplier[['Closure Form Filled Time']] = PIAmultiplier[['Closure Form Filled Time']].apply(changeDateTimeFormat)
deltaClosures = closures[closures['Closure Form Filled Time'] > PIAmultiplier['Closure Form Filled Time'].max()]
PIAmultiplier = pd.concat([PIAmultiplier,deltaClosures], ignore_index = True)
#PIAmultiplier = PIAmultiplier.append(deltaClosures)
#filtered_df = PIAmultiplier[PIAmultiplier['Closure Number'].notna() & (PIAmultiplier['Closure Number'] != '')]

In [ ]:
PIAmultiplier.shape

(4462, 137)

In [ ]:
# def col_idx_to_letter(idx):
#     """Convert a 1-based column index to a column letter."""
#     letter = ""
#     while idx > 0:
#         idx, remainder = divmod(idx - 1, 26)
#         letter = chr(65 + remainder) + letter
#     return letter


In [ ]:
#Write to sheet

spreadSheetId = "1DZPE9U2OZ9W5KwUIe0Vmj36yKHRSxT0CUSoKHvUxbIE"
rangeName = "Closures"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
df = pd.DataFrame(PIAmultiplier).iloc[:, :92]
# Clear only the necessary range
# num_rows, num_cols = df.shape
# end_column_letter = col_idx_to_letter(num_cols)
# clear_range = f"Closures!A1:{end_column_letter}{num_rows}"
# wb.values_clear(clear_range)
#Append data to sheet
set_with_dataframe(ws, df)

In [ ]:

sheet  = gc.open_by_url('https://docs.google.com/spreadsheets/d/1DZPE9U2OZ9W5KwUIe0Vmj36yKHRSxT0CUSoKHvUxbIE/edit#gid=0')
sheet =  sheet.get_worksheet(0)
PIAmultiplier = pd.DataFrame(sheet.get_all_records())
PIAmultiplier = PIAmultiplier[['Closure Number','Closure Date','Agreement Proof is valid (0/1)','Welfare amount if given specify amount','Closure Type as per PIA','Society Latitude as per PIA','Society Longitude as per PIA',
                               'Rent of the society as per PIA','New Devices as per HO','Refurbished Devices as per HO','VMS Sold Flats as per PIA','ERP Sold Flats As per PIA','Occupancy as per PIA','Occupancy % as per PIA',
                               'Competition Multiplier (PIA) - VMS','Competition Multiplier (PIA) - ERP']]

PIAmultiplier.rename({'Closure Number':'Closure Number', 'Closure Date':'Closure Date','Agreement Proof is valid (0/1)':'Agreement Validity','Welfare amount if given specify amount':'Welfare Amount - PIA',
                      'Closure Type as per PIA':'Closure Type - PIA','Society Latitude as per PIA':'Society Latitude - PIA','Society Longitude as per PIA':'Society Longitude - PIA',
                      'Rent of the society as per PIA':'Rent of the Society - PIA','New Devices as per HO':'New Devices - PIA','Refurbished Devices as per HO':'Refurbished Devices - PIA',
                      'VMS Sold Flats as per PIA':'VMS Sold Flats - PIA','ERP Sold Flats As per PIA':'ERP Sold Flats - PIA','Occupancy % as per PIA':'Occupancy % - PIA','Occupancy as per PIA':'Occupancy - PIA'
                      },inplace= True, axis = 1)

PIAmultiplier = PIAmultiplier[['Closure Number','Closure Date','Agreement Validity','Welfare Amount - PIA','Closure Type - PIA','Society Latitude - PIA','Society Longitude - PIA','Rent of the Society - PIA',
                               'New Devices - PIA','Refurbished Devices - PIA','VMS Sold Flats - PIA','ERP Sold Flats - PIA','Occupancy % - PIA','Occupancy - PIA','Competition Multiplier (PIA) - VMS',
                               'Competition Multiplier (PIA) - ERP']]

PIAmultiplier['Agreement Validity'] = pd.to_numeric(PIAmultiplier['Agreement Validity'],errors = 'coerce')
PIAmultiplier['Welfare Amount - PIA'] = pd.to_numeric(PIAmultiplier['Welfare Amount - PIA'],errors = 'coerce')
PIAmultiplier['Society Latitude - PIA'] = pd.to_numeric(PIAmultiplier['Society Latitude - PIA'],errors = 'coerce')
PIAmultiplier['Society Longitude - PIA'] = pd.to_numeric(PIAmultiplier['Society Longitude - PIA'],errors = 'coerce')
PIAmultiplier['Rent of the Society - PIA'] = pd.to_numeric(PIAmultiplier['Rent of the Society - PIA'],errors = 'coerce')
PIAmultiplier['VMS Sold Flats - PIA'] = pd.to_numeric(PIAmultiplier['VMS Sold Flats - PIA'],errors = 'coerce')
PIAmultiplier['ERP Sold Flats - PIA'] = pd.to_numeric(PIAmultiplier['ERP Sold Flats - PIA'],errors = 'coerce')
PIAmultiplier['Competition Multiplier (PIA) - VMS'] = pd.to_numeric(PIAmultiplier['Competition Multiplier (PIA) - VMS'],errors = 'coerce')
PIAmultiplier['Competition Multiplier (PIA) - ERP'] = pd.to_numeric(PIAmultiplier['Competition Multiplier (PIA) - ERP'],errors = 'coerce')

PIAmultiplier.loc[PIAmultiplier['Competition Multiplier (PIA) - VMS'].isna(),'Competition Multiplier (PIA) - VMS'] = 1
PIAmultiplier.loc[PIAmultiplier['Competition Multiplier (PIA) - ERP'].isna(),'Competition Multiplier (PIA) - ERP'] = 1
PIAmultiplier.loc[PIAmultiplier['Agreement Validity'].isna(), 'Agreement Validity'] = 0

PIAmultiplier = PIAmultiplier.sort_values(['Agreement Validity','Welfare Amount - PIA','Closure Type - PIA','Society Latitude - PIA','Society Longitude - PIA','Rent of the Society - PIA',
                'New Devices - PIA','Refurbished Devices - PIA','VMS Sold Flats - PIA','ERP Sold Flats - PIA','Occupancy % - PIA','Occupancy - PIA','Competition Multiplier (PIA) - VMS',
                'Competition Multiplier (PIA) - ERP'], ascending = False).groupby(['Closure Number','Closure Date']).head(1)
closures[['Closure Date']] = closures[['Closure Date']].apply(changeDateFormat)
PIAmultiplier[['Closure Date']] = PIAmultiplier[['Closure Date']].apply(changeDateFormat)

closures = pd.merge(closures, PIAmultiplier, on=['Closure Number','Closure Date'], how = 'left')

closures = closures.loc[closures['Closure Status']!="Marked Dummy"]
#closures = closures.loc[(closures['Team Category']!="FSD Team")]

In [ ]:
#FC Addon
spreadSheetId = "103g9RU2MX-vRBhousCKLE6XbCrVJG0KJFcHttDM2ht0"
rangeName = "FCAddon"
addon = get_data_df(spreadSheetId, rangeName)

In [ ]:
closures[['Closure Type','Closure Number','Closure Date']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 4460 entries, 0 to 4790
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Closure Type    4460 non-null   object
 1   Closure Number  4460 non-null   object
 2   Closure Date    4460 non-null   object
dtypes: object(3)
memory usage: 139.4+ KB


In [ ]:
addon[['Closure Type','Closure Number','Closure Date']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Closure Type    257 non-null    object
 1   Closure Number  257 non-null    object
 2   Closure Date    257 non-null    object
dtypes: object(3)
memory usage: 6.2+ KB


In [ ]:
closures['Closure Date'] = pd.to_datetime(closures['Closure Date'])
addon['Closure Date'] = pd.to_datetime(addon['Closure Date'])
addon['Closure Number'] = pd.to_numeric(addon['Closure Number'],errors  = 'coerce')
addon['Addon VMS FC'] = pd.to_numeric(addon['Addon VMS FC'],errors  = 'coerce')
addon['Addon ERP FC'] = pd.to_numeric(addon['Addon ERP FC'],errors  = 'coerce')

In [ ]:

closures[['Closure Type','Closure Number','Closure Date']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 4460 entries, 0 to 4790
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Closure Type    4460 non-null   object        
 1   Closure Number  4460 non-null   object        
 2   Closure Date    4460 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 139.4+ KB


In [ ]:
addon[['Closure Type','Closure Number','Closure Date']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Closure Type    257 non-null    object        
 1   Closure Number  257 non-null    int64         
 2   Closure Date    257 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 6.2+ KB


In [ ]:
closures = pd.merge(closures, addon[['Closure Number','Closure Date','Closure Type','Addon VMS FC','Addon ERP FC']],on=['Closure Number','Closure Date','Closure Type'], how = 'left')

In [ ]:

closures['Closure Date'] = pd.to_datetime(closures['Closure Date']).dt.date

In [ ]:
closures[closures['Closure Type'] == 'Addons'][['Closure Date','Closure Number','Closure Type','Addon VMS FC','Addon ERP FC']]

,Closure Date,Closure Number,Closure Type,Addon VMS FC,Addon ERP FC
150,2023-10-26,1032847,Addons,NaN,NaN
475,2023-11-30,1026447,Addons,NaN,NaN
477,2023-11-30,1020338,Addons,NaN,NaN
478,2023-11-30,1029390,Addons,NaN,NaN
479,2023-11-30,1028003,Addons,NaN,NaN
...,...,...,...,...,...
4434,2025-02-26,1020058,Addons,NaN,NaN
4435,2025-02-26,1008126,Addons,NaN,NaN
4436,2025-02-26,1020599,Addons,NaN,NaN
4437,2025-02-27,1013815,Addons,NaN,NaN


In [ ]:
closures.loc[(closures['Closure Type'] == 'Addons') & (closures['Closure Date'] > pd.to_datetime("26 Jan 2024").date()), 'VMS Sold Flats - PIA'] = closures['Addon VMS FC']
closures.loc[(closures['Closure Type'] == 'Addons') & (closures['Closure Date'] > pd.to_datetime("26 Jan 2024").date()), 'ERP Sold Flats - PIA'] = closures['Addon ERP FC']

In [ ]:
closures = closures[['Closure Type','Closure Date','Original Closure Number','Closure Number','Closure Form Filled Date','Closure Form Filled Time','Closure Month','Closure Form Filled By','Team Type','Lead Type',
                     'Closure Done By (Primary Email)','Primary BD City','Primary BD Role','Closure Done By (Secondary Email)','Secondary BD City','Secondary BD Role','Society Type','Property Type','VMSTrialPeriod',
                     'ERPTrialPeriod','Society Name (Sales)','Closure City','City Type','Sales Product','HO Product','Total Flats','VMS Sold Flats','ERP Sold Flats','Sold Flats','Occupancy Percentage','Pin Code',
                     'Society Latitude','Society Longitude','Rent of the society','New Devices','Refurbished Devices','Subscription Revenue','Upfront Revenue (One time)','Access control/Boom Barrier net revenue',
                     'Welfare Amount','Monetization Advance Amount','RP (Revenue Points) to be claimed','Revenue to be claimed','WrongPrimaryBDMail','WrongSecondaryBDMail','WrongHOProduct','CrossCityClosure',
                     'DuplicateClosureNumber','WrongOriginalClosureNumber','ClosureActivity','VMSHOFilledDate','VMSHOFilledEmail','VMSHOMeetingID','VMSHODate','Occupied Flats (VMS)','Sold Flats (VMS)','VMSHOStatus',
                     'VMS Handover Type','Obstruction in VMS HO','VMS SKU ID','VMS HO Done by','ERPHOFilledDate','ERPHOFilledEmail','ERPHOMeetingID','ERPHODate','Occupied Flats (ERP)','Sold Flats (ERP)','ERPHOStatus',
                     'ERP Handover Type','Obstruction in ERP HO','ERP SKU ID','ERP HO Done by','VMS Sold Flats Mismatch','VMSHOClosureFilledTAT','VMSHOClosureTAT','ERP Sold Flats Mismatch','ERPHOClosureFilledTAT',
                     'ERPHOClosureTAT','Closure Form Filling TAT','monthDays','year','HO Meeting ID','Current VMS software used','Current ERP software used','Competition Proof Link (VMS)','Competition Proof Link (ERP)',
                     'Closure Proof Link','Agreement Type','Closure Status','Team Category','Last Updated Datetime','VMS Sold Flats as per Occupancy','Brand New Devices','Biometrics Devices','Agreement Validity','Welfare Amount - PIA',
                     'Closure Type - PIA','Society Latitude - PIA','Society Longitude - PIA','Rent of the Society - PIA','New Devices - PIA','Refurbished Devices - PIA','VMS Sold Flats - PIA','ERP Sold Flats - PIA',
                     'Occupancy % - PIA','Occupancy - PIA','Competition Multiplier (PIA) - VMS','Competition Multiplier (PIA) - ERP']]

In [ ]:

## Appending data for RP_D1 Calculation
spreadSheetId="1SqiNSq8YUm61Ai3OKPV4QpRBOTNztrVYG2AkO3jZwe4"
rangeName="RP_HO"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
#ws.clear()
wb.values_clear("RP_HO!A5:DD")
row = 5
col = 1
df = pd.DataFrame(closures)
set_with_dataframe(ws, df,row = row,col=col)


### Read Data from RP_D4 Calculation
def get_data_df_exp(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[5:], columns = l[4])
  return ldf
spreadSheetId = "1SqiNSq8YUm61Ai3OKPV4QpRBOTNztrVYG2AkO3jZwe4"
rangeName = "RP_D4"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
RP_D4 = get_data_df_exp(spreadSheetId, rangeName)

RP_D4 = RP_D4[['Closure Number','Closure Date','New device cost',	'Refurbished device cost',	'Monetization cost',	'Sales cost','Intercept Value','Flat Count as per occupancy','Flat Count Value','Locality','Locality value',
               'Premiumness','Percentage of median rent','Premiumness Value',	"Revenue (Upfront, Subscription, Access Control/ Boom Barrier)",'Society RP as per Sales Product','RP_D4','Society RP Eligibility',
               'Society RP as per HO Product','RP_HO','Gross RP Without Offset']]

closures['Closure Number'] = pd.to_numeric(closures['Closure Number'],errors = 'coerce')
RP_D4['Closure Number'] = pd.to_numeric(RP_D4['Closure Number'],errors = 'coerce')
closures[['Closure Date']] = closures[['Closure Date']].apply(changeDateFormat)
RP_D4[['Closure Date']] = RP_D4[['Closure Date']].apply(changeDateFormat)

closures = pd.merge(closures, RP_D4, on=['Closure Number','Closure Date'], how = 'left')

In [ ]:
### Read Data from RP_HO Calculation
def get_data_df_exp(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[5:], columns = l[4])
  return ldf
spreadSheetId = "1SqiNSq8YUm61Ai3OKPV4QpRBOTNztrVYG2AkO3jZwe4"
rangeName = "RP_HO"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
RP_HO = get_data_df_exp(spreadSheetId, rangeName)

In [ ]:
RP_HO = RP_HO[['Closure Number','Closure Date','Flat Count as per occupancy - PIA','Flat Count Value - PIA','Locality - PIA','Locality Value - PIA','Premiumness - PIA',
               'Percentage of median rent - PIA','Premiumness Value - PIA','Society RP as per D1 Product','RP D1','Gross RP - PIA_Validated RP Component','Gross RP - PIA_Only Valid',
               'Gross RP - PIA as per D1 Product','Gross RP - PIA_Validated RP Component Onboarded','VMS Onb Status','ERP Onb Status','Onb Product_Current Signing']]

closures['Closure Number'] = pd.to_numeric(closures['Closure Number'],errors = 'coerce')
RP_HO['Closure Number'] = pd.to_numeric(RP_HO['Closure Number'],errors = 'coerce')
RP_HO['RP D1'] = pd.to_numeric(RP_HO['RP D1'],errors = 'coerce')
RP_HO['Gross RP - PIA_Validated RP Component'] = pd.to_numeric(RP_HO['Gross RP - PIA_Validated RP Component'],errors = 'coerce')
RP_HO['Gross RP - PIA_Only Valid'] = pd.to_numeric(RP_HO['Gross RP - PIA_Only Valid'],errors = 'coerce')
RP_HO['Gross RP - PIA as per D1 Product'] = pd.to_numeric(RP_HO['Gross RP - PIA as per D1 Product'],errors = 'coerce')
RP_HO['Gross RP - PIA_Validated RP Component Onboarded'] = pd.to_numeric(RP_HO['Gross RP - PIA_Validated RP Component Onboarded'],errors = 'coerce')

closures[['Closure Date']] = closures[['Closure Date']].apply(changeDateFormat)
RP_HO[['Closure Date']] = RP_HO[['Closure Date']].apply(changeDateFormat)

closures = pd.merge(closures, RP_HO, on=['Closure Number','Closure Date'], how = 'left')

In [ ]:
closures = closures[["Closure Type", "Closure Date", "Original Closure Number", "Closure Number", "Closure Form Filled Date", "Closure Form Filled Time", "Closure Month",
                     "Closure Form Filled By", "Team Type", "Lead Type", "Closure Done By (Primary Email)", "Primary BD City", "Primary BD Role", "Closure Done By (Secondary Email)",
                     "Secondary BD City", "Secondary BD Role", "Society Type", "Property Type", "VMSTrialPeriod", "ERPTrialPeriod", "Society Name (Sales)", "Closure City", "City Type",
                     "Sales Product", "HO Product", "Total Flats", "VMS Sold Flats", "ERP Sold Flats", "Sold Flats", "Occupancy Percentage", "Pin Code", "Society Latitude", "Society Longitude",
                     "Rent of the society", "New Devices", "Refurbished Devices",'Brand New Devices','Biometrics Devices', "Subscription Revenue", "Upfront Revenue (One time)", "Access control/Boom Barrier net revenue",
                     "Welfare Amount", "Monetization Advance Amount", "RP (Revenue Points) to be claimed", "Revenue to be claimed", "WrongPrimaryBDMail", "WrongSecondaryBDMail", "WrongHOProduct",
                     "CrossCityClosure", "DuplicateClosureNumber", "WrongOriginalClosureNumber", "ClosureActivity", "VMSHOFilledDate", "VMSHOFilledEmail", "VMSHOMeetingID", "VMSHODate",
                     "Occupied Flats (VMS)", "Sold Flats (VMS)", "VMSHOStatus", "VMS Handover Type", "Obstruction in VMS HO", "VMS SKU ID", "VMS HO Done by", "ERPHOFilledDate", "ERPHOFilledEmail",
                     "ERPHOMeetingID", "ERPHODate", "Occupied Flats (ERP)", "Sold Flats (ERP)", "ERPHOStatus", "ERP Handover Type", "Obstruction in ERP HO", "ERP SKU ID", "ERP HO Done by",
                     "VMS Sold Flats Mismatch", "VMSHOClosureFilledTAT", "VMSHOClosureTAT", "ERP Sold Flats Mismatch", "ERPHOClosureFilledTAT", "ERPHOClosureTAT", "Closure Form Filling TAT",
                     "monthDays", "year", "HO Meeting ID", "Current VMS software used", "Current ERP software used", "Closure Proof Link", "Agreement Type", "Closure Status",
                     "Team Category", "New device cost", "Refurbished device cost", "Monetization cost", "Sales cost", "Intercept Value", "Flat Count as per occupancy", "Flat Count Value",
                     "Locality", "Locality value", "Premiumness", "Percentage of median rent", "Premiumness Value", "Revenue (Upfront, Subscription, Access Control/ Boom Barrier)",
                     "Agreement Validity", "Welfare Amount - PIA", "Closure Type - PIA", "Society Latitude - PIA", "Society Longitude - PIA", "Rent of the Society - PIA", "New Devices - PIA",
                     "Refurbished Devices - PIA", "VMS Sold Flats - PIA", "ERP Sold Flats - PIA", "Occupancy % - PIA", "Occupancy - PIA","Flat Count as per occupancy - PIA", "Flat Count Value - PIA",
                     "Locality - PIA", "Locality Value - PIA", "Premiumness - PIA", "Percentage of median rent - PIA", "Premiumness Value - PIA",'Competition Multiplier (PIA) - VMS','Competition Multiplier (PIA) - ERP',
                     "Society RP Eligibility","Society RP as per Sales Product", "RP_D4", "Society RP as per HO Product","RP_HO" ,"RP D1",'Gross RP Without Offset','VMS Sold Flats as per Occupancy',
                     'Gross RP - PIA_Validated RP Component', 'Gross RP - PIA_Only Valid', 'Gross RP - PIA as per D1 Product','Gross RP - PIA_Validated RP Component Onboarded',
                     'VMS Onb Status','ERP Onb Status','Onb Product_Current Signing']]

In [ ]:
def GrossRP_multiplier(row):
    if row['Closure City'] in ['Mumbai']:
        if pd.Timestamp(row['Closure Date']) > pd.to_datetime("27 Mar 2024") and pd.Timestamp(row['Closure Date']) <= pd.to_datetime("26 May 2024"):
            if row['Sold Flats'] <= 50:
                return 0.5
            elif row['Sold Flats'] > 50 and row['Sold Flats'] <= 200:
                return 2.5
            elif row['Sold Flats'] > 200:
                return 0.75
        elif pd.Timestamp(row['Closure Date']) > pd.to_datetime("26 May 2024") and pd.Timestamp(row['Closure Date']) <= pd.to_datetime("26 Sep 2024"):
            if row['Sold Flats'] <= 50:
                return 0.5
            elif row['Sold Flats'] > 50 and row['Sold Flats'] <= 200:
                return 2.85
            elif row['Sold Flats'] > 200:
                return 0.9
    return 1

closures['Gross RP Multiplier'] = closures.apply(GrossRP_multiplier, axis=1)

closures['Gross RP Without Offset'] = pd.to_numeric(closures['Gross RP Without Offset']) * closures['Gross RP Multiplier']
closures['Gross RP - PIA_Validated RP Component'] = pd.to_numeric(closures['Gross RP - PIA_Validated RP Component']) * closures['Gross RP Multiplier']
closures['Gross RP - PIA_Only Valid'] = pd.to_numeric(closures['Gross RP - PIA_Only Valid']) * closures['Gross RP Multiplier']
closures['Gross RP - PIA as per D1 Product'] = pd.to_numeric(closures['Gross RP - PIA as per D1 Product']) * closures['Gross RP Multiplier']

In [ ]:
closures['Gross RP Multiplier'].value_counts()

,count
Gross RP Multiplier,
1.00,4359
2.85,33
2.50,29
0.50,26
0.90,14
0.75,5


In [ ]:

# Append PIA Validation
spreadSheetId="1DZPE9U2OZ9W5KwUIe0Vmj36yKHRSxT0CUSoKHvUxbIE"
rangeName="Validation Master"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
ws.clear()
df = pd.DataFrame(PIAmultiplier)
set_with_dataframe(ws, df)

In [ ]:
closures[closures['Closure Number']==1038800][['Closure Number','Closure Date', 'Sales Product','HO Product','RP_D4','RP_HO',"RP D1"]]

,Closure Number,Closure Date,Sales Product,HO Product,RP_D4,RP_HO,RP D1
3641,1038800,2025-01-15,VMS only,VMS only,299705.89,299705.89,300984.699


#**ONB Map**

In [ ]:
#Final Master Data Autobackups

dump_name = 'Final Master Data'
Inventory = pd.read_csv(file_path + DumpDate + "/Master/"+ dump_name +".csv",parse_dates=True)

Inventory.rename({'kibanaID':'Kibana ID','closureID.Uniview':'Closure Number','closureID.Original':'Original Closure Number'}, inplace = True, axis = 1)
kibanaMap = Inventory[['Kibana ID','Closure Number','dateSigning']]
kibanaMap.loc[:,'Closure Number'] = pd.to_numeric(kibanaMap['Closure Number'], errors = 'coerce')
kibanaMap = kibanaMap[~kibanaMap['Closure Number'].isna()]
kibanaMap = kibanaMap.sort_values('dateSigning').groupby(['Closure Number']).tail(1)
kibanaMap = kibanaMap[['Kibana ID','Closure Number']]

closures = pd.merge(closures, kibanaMap, on = ['Closure Number'], how = 'left')

<ipython-input-139-617a80b7f66f>:4: DtypeWarning: Columns (10,17,18,20,34,51) have mixed types. Specify dtype option on import or set low_memory=False.
  Inventory = pd.read_csv(file_path + DumpDate + "/Master/"+ dump_name +".csv",parse_dates=True)
<ipython-input-139-617a80b7f66f>:13: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  closures = pd.merge(closures, kibanaMap, on = ['Closure Number'], how = 'left')


In [ ]:
#################################

dump_name = 'Society Creation Date - Insights Copy'
society_creation_date = pd.read_csv(file_path + DumpDate + "/Sai/"+ dump_name +".csv",parse_dates=True)

society_creation_date= society_creation_date[['id', 'name','created_on','closure_number','latitude','longitude']]
society_creation_date.rename({'id':'Kibana ID','created_on':'Kibana Creation Date','name':'Kibana Name','closure_number':'Closure Number'},inplace=True, axis=1)
society_creation_date[['Kibana Creation Date']] = society_creation_date[['Kibana Creation Date']].apply(changeDateFormat)

society_creation_date.loc[:,'Closure Number'] = pd.to_numeric(society_creation_date['Closure Number'], errors = 'coerce')
society_creation_date[['Kibana Name']] = society_creation_date[['Kibana Name']].apply(cleanText)
society_creation_date = society_creation_date.sort_values('Kibana Creation Date').groupby(['Kibana Name']).tail(1)
society_creation_date = society_creation_date[society_creation_date['Kibana Creation Date'] >= pd.to_datetime('1 July 2021').date()]
society_creation_date['Lat_Long'] = society_creation_date['latitude'].astype(str)+","+society_creation_date['longitude'].astype(str)

society_creation_date.rename({'Kibana ID':'Auto-Matched Kibana ID'}, inplace = True, axis = 1)

soc_latlong = society_creation_date[['Closure Number','Lat_Long']]

closures = pd.merge(closures, soc_latlong, left_on = 'Closure Number', right_on = 'Closure Number', how = 'left')

socnames = society_creation_date[['Kibana Name']].drop_duplicates()

closures = pd.merge(closures, society_creation_date[['Closure Number','Kibana Name','Auto-Matched Kibana ID']], on = 'Closure Number', how = 'left')

# def textMatch(row):
#   ratio = process.extract(row['Society Name (Sales)'], socnames['Kibana Name'] , limit = 1 , scorer = fuzz.ratio)
#   if (ratio[0][1] >= 95):
#     return ratio[0][0]

# closures['Possible Kibana Name Match'] = closures.apply(textMatch, axis = 1)

# closures = pd.merge(closures, society_creation_date[['Auto-Matched Kibana ID','Kibana Name']], left_on = 'Possible Kibana Name Match', right_on = 'Kibana Name', how = 'left')

In [ ]:
spreadSheetId="1aVAu23rPNHyOEk2ioNXdbqyUKEFzcXedlY283cvDKvY"
rangeName="Society Master"
soc_master=get_data_df(spreadSheetId, rangeName)

soc_master_vms = soc_master[['Closure Number (VMS)','VMS Onb Status','VMSOnbDate']]
soc_master_vms.rename({'Closure Number (VMS)':'Closure Number','VMS Onb Status':'Onb Status','VMSOnbDate':'Onb Date'},inplace=True, axis=1)
soc_master_vms = soc_master_vms.loc[(soc_master_vms['Onb Status']!="NOT SOLD")]

soc_master_erp = soc_master[['Closure Number (ERP)','ERP Onb Status','ERPOnbDate']]
soc_master_erp.rename({'Closure Number (ERP)':'Closure Number','ERP Onb Status':'Onb Status','ERPOnbDate':'Onb Date'},inplace=True, axis=1)
soc_master_erp = soc_master_erp.loc[(soc_master_erp['Onb Status']!="NOT SOLD")]

# soc_master_vms.loc[soc_master_vms['Onb Status'].isin(['YTB','PHASE 1 CHURN','PHASE 2 CHURN','ON HOLD']),'Onb Date']=soc_master_erp['Onb Date']
# soc_master_vms.loc[soc_master_vms['Onb Status'].isin(['YTB','PHASE 1 CHURN','PHASE 2 CHURN','ON HOLD']),'Onb Status']=soc_master_erp['Onb Status']

# soc_master_erp.loc[soc_master_erp['Onb Status'].isin(['YTB','PHASE 1 CHURN','PHASE 2 CHURN','ON HOLD']),'Onb Date']=soc_master_vms['Onb Date']
# soc_master_erp.loc[soc_master_erp['Onb Status'].isin(['YTB','PHASE 1 CHURN','PHASE 2 CHURN','ON HOLD']),'Onb Status']=soc_master_vms['Onb Status']

onb_merge = pd.concat([soc_master_vms,soc_master_erp],axis=0)
onb_merge = onb_merge[onb_merge['Closure Number']!='']
onb_merge.loc[:,'Closure Number'] = pd.to_numeric(onb_merge['Closure Number'], errors = 'coerce')
onb_merge[['Onb Date']] = onb_merge[['Onb Date']].apply(changeDateFormat)

#onb_grp = onb_merge.groupby(['Closure Number','Onb Status']).agg({'Onb Date':'max'}).reset_index()

onb_merge = onb_merge.drop_duplicates(subset = ['Closure Number'],keep = 'first')

closures = pd.merge(closures, onb_merge, on = 'Closure Number', how = 'left')

<ipython-input-141-ba7d2cf2ee07>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soc_master_vms.rename({'Closure Number (VMS)':'Closure Number','VMS Onb Status':'Onb Status','VMSOnbDate':'Onb Date'},inplace=True, axis=1)
<ipython-input-141-ba7d2cf2ee07>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soc_master_erp.rename({'Closure Number (ERP)':'Closure Number','ERP Onb Status':'Onb Status','ERPOnbDate':'Onb Date'},inplace=True, axis=1)


In [ ]:
onb_merge[onb_merge['Closure Number']==1031054]

,Closure Number,Onb Status,Onb Date
5459,1031054.0,ON-BOARDED,2023-04-30


In [ ]:
closures[~closures['Kibana ID'].isna()].shape

(3925, 146)

In [ ]:
'''
# Fuzzy Match of Society Name and Original closure number :
df1 = soc_master[['Original Closure Number', 'Society Name']]
df2 = closures[['Closure Type','Closure Date','Original Closure Number','Closure Number', 'Society Name (Sales)','Closure City', 'Sales Product', 'HO Product']]
df2 = df2[(df2['Closure Type']=='Upsell (VMS/ERP)') | (df2['Closure Type']=='Winback')]

df1['key'] = df1['Original Closure Number'].astype(str) + '-' + df1['Society Name']
df2['key'] = df2['Original Closure Number'].astype(str) + '-' + df2['Society Name (Sales)']

#fuzzywuzzy UDF code/ use fuzz.token_set_ratio for positional consideration
def find_match(row):
    match = process.extractOne(row['key'], df1['key'], scorer=fuzz.token_sort_ratio)
    matched_key = df1.loc[match[2], 'key']
    similarity_score = match[1]
    return pd.Series([matched_key,similarity_score], index=['Matched_key', 'Similarity_Score'])

#1st part
st=time.time()
# Apply the find_match function to the entire 'name' column in df1
df2[['Matched_key', 'Similarity_Score']] = df2.apply(find_match, axis=1)
et=time.time()

elapsed_time_minutes = (et - st) / 60
# Print the elapsed time in minutes with 4 decimal places
print(f"Elapsed time: {elapsed_time_minutes:.4f} minutes")

df2.loc[(df2['Closure Type'].isin({'Upsell (VMS/ERP)','Winback'})) & (df2['Similarity_Score'] < 95),'WrongOriginalClosure_SocietyName'] = True

spreadSheetId = "144t7Cm1lXYHP4I-gCi_ZZH84imNtGFBNzZSY5Xf57V8"
wb = gc.open_by_key(spreadSheetId)

ws = wb.worksheet("Base")
ws.clear()

# APPEND DATA TO SHEET
set_with_dataframe(ws, df2)
'''

'\n# Fuzzy Match of Society Name and Original closure number :\ndf1 = soc_master[[\'Original Closure Number\', \'Society Name\']]\ndf2 = closures[[\'Closure Type\',\'Closure Date\',\'Original Closure Number\',\'Closure Number\', \'Society Name (Sales)\',\'Closure City\', \'Sales Product\', \'HO Product\']]\ndf2 = df2[(df2[\'Closure Type\']==\'Upsell (VMS/ERP)\') | (df2[\'Closure Type\']==\'Winback\')]\n\ndf1[\'key\'] = df1[\'Original Closure Number\'].astype(str) + \'-\' + df1[\'Society Name\']\ndf2[\'key\'] = df2[\'Original Closure Number\'].astype(str) + \'-\' + df2[\'Society Name (Sales)\']\n\n#fuzzywuzzy UDF code/ use fuzz.token_set_ratio for positional consideration\ndef find_match(row):\n    match = process.extractOne(row[\'key\'], df1[\'key\'], scorer=fuzz.token_sort_ratio)\n    matched_key = df1.loc[match[2], \'key\']\n    similarity_score = match[1]\n    return pd.Series([matched_key,similarity_score], index=[\'Matched_key\', \'Similarity_Score\'])\n\n#1st part\nst=time.time()

#Ops Parameter Merge

#**📕Tech Dump**

In [ ]:
dump_name = 'Society Creation Date - Insights Copy'
society_creation_date = pd.read_csv(file_path + DumpDate + "/Sai/"+ dump_name +".csv",parse_dates=True)

society_creation_date= society_creation_date[['id','created_on']]
society_creation_date.rename({'id':'Kibana ID','created_on':'Kibana Creation Date'},inplace=True, axis=1)
society_creation_date[['Kibana Creation Date']] = society_creation_date[['Kibana Creation Date']].apply(changeDateFormat)
society_creation_date = society_creation_date[society_creation_date['Kibana Creation Date'] >= pd.to_datetime("01 July 2023").date()]

###################################################### it can be removed

dump_name = 'ERP Enabled Date'
erp_activation = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)

erp_activation.rename({'society_id':'Kibana ID','created_on':'ERP Activation Date'},inplace=True, axis=1)
erp_activation[['ERP Activation Date']] = erp_activation[['ERP Activation Date']].apply(changeDateFormat)
erp_activation=erp_activation.sort_values('ERP Activation Date').groupby('Kibana ID').tail(1)

######################################################

dump_name = 'Bank account added and their status'
account = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)

account[['created_on']]= account[['created_on']].apply(changeDateFormat)
account = account[['society_id','created_on','account_status']]
account = account.sort_values(['created_on']).groupby(['society_id']).tail(1)
account.rename({'society_id':'Kibana ID','created_on':'Bank Addition Date','account_status':'Society AccountStatus'},axis=1,inplace=True)


######################################################

dump_name = 'onboarding_dashboard_metrics'   ### True Downloads with PAC - Insights Copy 4 March

Download = pd.read_csv(file_path + DumpDate + "/Sai/"+ dump_name +".csv",parse_dates=True)
Download.rename({'Society_id':'Kibana ID','True_downloads':'Download Count'},inplace=True,axis=1)
Download = Download[['Kibana ID','Download Count']]

######################################################

dump_name = 'Last 30 day transactions - Hood Insights - Last Updated date'
Transaction = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)
Transaction.rename({'Last_Updated_Date':'Payment Date'},inplace=True,axis=1)
Transaction[['Payment Date']] = Transaction[['Payment Date']].apply(changeDateFormat)
#masterTransaction = masterTransaction[masterTransaction['Payment Date'] >= pd.to_datetime("01 July 2023").date()]

dump_name = 'TransactionsSep23-Mar24'
transaction_v2 = pd.read_csv("/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/TransactionMaster/"+ dump_name +".csv")
transaction_v2.rename({'Last_Updated_Date':'Payment Date'},inplace=True,axis=1)
transaction_v2[['Payment Date']] = transaction_v2[['Payment Date']].apply(changeDateFormat)

masterTransaction = pd.concat([transaction_v2,Transaction], ignore_index = True)

masterTransaction = masterTransaction[['Society ID','Unique Flats', 'Transactions', 'Amount','Online Offline Mode','Payment Date']]
masterTransaction.rename({'Society ID':'Kibana ID','Transactions':'Payments Count','Unique Flats':'Payment Unique Flats',
                          'Amount':'Total Payment Amount in K','Online Offline Mode':'Payment Mode'},inplace=True,axis=1)

masterTransaction['Total Payment Amount in K'] = masterTransaction['Total Payment Amount in K'].astype(str).str.replace(",", "")
#masterTransaction['Total Payment Amount in K'] = masterTransaction['Total Payment Amount in K'].str.replace(",", "")
masterTransaction['Total Payment Amount in K'] = pd.to_numeric(masterTransaction['Total Payment Amount in K'])

masterTransaction['Payment Unique Flats'] = masterTransaction['Payment Unique Flats'].astype(int)


masterTransaction['Total Payment Amount in K'] = (masterTransaction['Total Payment Amount in K']/1000)

masterTransaction[['Payment Date']] = masterTransaction[['Payment Date']].apply(changeDateFormat)

masterTransaction = masterTransaction[masterTransaction['Payment Mode'] == 'Online']
masterTransaction = masterTransaction[masterTransaction['Payment Date'] >= pd.to_datetime('1 September 2023').date()]

masterTransaction = masterTransaction.groupby('Kibana ID').agg({'Payment Unique Flats':'sum','Total Payment Amount in K':'sum'}).reset_index()

######################################################

dump_name = 'Bill Publish Date Final'
bpd = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)
bpd = bpd[bpd['Bill Type'] == 'Recurring']

bpd=bpd[[ 'Society ID', 'Start Date', 'End Date', 'Due Date',
        'Approved date', 'Bill Types', 'Bill Frequency', 'Flat Count', 'Total Amount']]
bpd.rename({'Approved date':'Approved Date'},inplace=True, axis=1)

bpd[['Start Date','End Date','Due Date','Approved Date']] = bpd[['Start Date','End Date','Due Date','Approved Date']].apply(changeDateFormat)

######################################################

dump_name = 'ERP - Deleted Cancelled Bills Report'
deleted = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)

deleted.rename({'Deletion/Cancellation Date':'Deleted Date','Bill Type':'Bill Types','Bill Count':'Valid Bill Count','Status':'Deletion Status'},inplace=True,axis=1)
deleted=deleted[['Society ID',  'Start Date', 'End Date','Due Date',
                 'Approved Date', 'Bill Types','Bill Frequency','Flat Count','Deleted Date','Deletion Status','Approval Status']]

deleted[['Start Date','End Date','Due Date','Approved Date','Deleted Date']] = deleted[['Start Date','End Date','Due Date','Approved Date','Deleted Date']].apply(changeDateFormat)

deleted=deleted[deleted['Approval Status']=='Approved']


#bpd_deleted=bpd.append(deleted)
bpd_deleted = pd.concat([bpd,deleted],ignore_index = True)
bpd_deleted['Deletion Status']=bpd_deleted['Deletion Status'].fillna('Not Deleted')
bpd_deleted['Bill Types']=bpd_deleted['Bill Types'].str.lower()

def bill_type_clean(row):
    if 'maint' in row['Bill Types']:
        return 'MaintainanceAUTOTAG'
    elif 'mait' in row['Bill Types']:
        return 'MaintainanceAUTOTAG'
    elif 'MAINT' in row['Bill Types']:
        return 'MaintainanceAUTOTAG'
    elif 'maiint' in row['Bill Types']:
        return 'MaintainanceAUTOTAG'
    elif 'cam' in row['Bill Types']:
        return 'MaintainanceAUTOTAG'
    elif 'e_m' in row['Bill Types']:
        return 'MaintainanceAUTOTAG'
    elif 'park' in row['Bill Types']:
        return "ParkingChargesAUTOTAG"
    elif 'water' in row['Bill Types']:
        return 'WaterAUTOTAG'
    elif 'electricity' in row['Bill Types']:
        return 'ElectricityAUTOTAG'
    elif 'utility' in row['Bill Types']:
        return 'UtilityAUTOTAG'
    elif 'gas' in row['Bill Types']:
        return 'GasAUTOTAG'
    else:
        return row['Bill Types']

bpd_deleted['Bill Types']=bpd_deleted.apply(bill_type_clean,axis=1)

def cleanBillFrequency(row):
    if row['Bill Frequency']>0 and row['Bill Frequency']<=12:
        return 12/12
    elif row['Bill Frequency']>20 and row['Bill Frequency']<=24:
        return 12/4
    elif row['Bill Frequency']>26 and row['Bill Frequency']<=28:
        return 12/2
    else:
        return 12/1

bpd_deleted['Bill Frequency']=bpd_deleted.apply(cleanBillFrequency,axis=1)

bpd_deleted=bpd_deleted[['Society ID', 'Approved Date', 'Deleted Date','Deletion Status',
                         'Bill Frequency','Bill Types', 'Start Date', 'End Date', 'Due Date','Flat Count', 'Total Amount']]

#bpd_deleted['Total Amount']=bpd_deleted['Total Amount'].str.replace(',','')
bpd_deleted['Total Amount'] = bpd_deleted['Total Amount'].astype(float)

bpd_deleted.rename({'Society ID':'Kibana ID','Approved Date':'Posted Date', 'Total Amount':'Total Posted Amount in K'},inplace=True,axis=1)

bpd_deleted['Total Posted Amount in K'] = (bpd_deleted['Total Posted Amount in K']/1000).round(0)

######################################################

bpd_deleted = bpd_deleted[bpd_deleted['Posted Date'] >= pd.to_datetime('1 September 2023').date()]

<ipython-input-129-0114eda36bd8>:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(row, errors='coerce').dt.date
<ipython-input-145-8bf239941b74>:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bpd_deleted['Total Amount'] = bpd_deleted['Total Amount'].astype(float)
<ipython-input-145-8bf239941b74>:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bpd_deleted.rename({'Society ID':'Kibana ID','Approved

In [ ]:
pd.to_datetime(Transaction['Payment Date']).min()

Timestamp('2024-09-01 00:00:00')

#**Posting Calculations**

In [ ]:
masterKibana = reduce(lambda left,right: pd.merge(left,right, on=['Kibana ID'], how='outer'),[society_creation_date,account,masterTransaction])
masterTechPosting = pd.merge(masterKibana, bpd_deleted, left_on = ['Kibana ID'], right_on = ['Kibana ID'], how = 'left')
masterTechPosting = pd.merge(masterTechPosting, closures[['Kibana ID', 'Sold Flats']], left_on = ['Kibana ID'], right_on = ['Kibana ID'], how = 'right')
masterTechPosting.drop_duplicates(inplace = True)
masterTechPosting = masterTechPosting[~(masterTechPosting['Kibana ID'].isna())]

masterERPPosting = masterTechPosting.copy()
# masterERPPosting['Sold Flats'].fillna(masterERPPosting['RDcount'], inplace=True)
masterERPPosting = masterERPPosting[~(masterERPPosting['Sold Flats'].isna())]
# masterERPPosting['RDperc'] = (masterERPPosting['RDcount']/masterERPPosting['Sold Flats']*100)
#masterERPPosting['Flat Count'] = masterERPPosting['Flat Count'].str.replace(',','')
masterERPPosting['Flat Count'] = pd.to_numeric(masterERPPosting['Flat Count'])
masterERPPosting['Postingperc'] = (masterERPPosting['Flat Count']/masterERPPosting['Sold Flats']*100)
masterERPPosting['TransactionPerc'] = (masterERPPosting['Payment Unique Flats']/masterERPPosting['Sold Flats']*100)

In [ ]:
conditions = [(masterERPPosting['Postingperc'].isna()),
              (masterERPPosting['Postingperc'] <= 40)]
values = ['No Posting','Flat Posting IPR']
masterERPPosting['Flat Posting IPR'] = np.select(conditions, values)

conditions = [(masterERPPosting['Society AccountStatus'] != 'VERIFIED')]
values = ['Account IPR']
masterERPPosting['Account IPR'] = np.select(conditions, values)

conditions = [(masterERPPosting['TransactionPerc'] <= 10)]
values = ['Transaction IPR']
masterERPPosting['Transaction IPR'] = np.select(conditions, values)

cols = ['Flat Posting IPR', 'Account IPR', 'Transaction IPR']
masterERPPosting['combined IPR'] = masterERPPosting[cols].apply(lambda row: '||'.join(row.values.astype(str)), axis=1)
IPR_condition = masterERPPosting[['Kibana ID', 'combined IPR']]
masterERPPosting['Start Date1'] = pd.to_datetime(masterERPPosting[~masterERPPosting['Start Date'].isna()]['Start Date']).apply(lambda x: str(x.to_period('M')))
masterERPPosting['End Date1'] = pd.to_datetime(masterERPPosting[~masterERPPosting['End Date'].isna()]['End Date']).apply(lambda x: str(x.to_period('M')))
masterERPPosting['Bill Period'] = (masterERPPosting['Start Date1']) + ' to ' + (masterERPPosting['End Date1'])

def diff_month(row):
  return (row['End Date'].year - row['Start Date'].year) * 12 + row['End Date'].month - row['Start Date'].month  + int(round((row['End Date'].day - row['Start Date'].day)/28))

masterERPPosting['Bill Cycles Posted']=(masterERPPosting[~((masterERPPosting['End Date'].isna()) | (masterERPPosting['Start Date'].isna()))].apply(diff_month,axis=1))/masterERPPosting['Bill Frequency']
masterERPPosting.drop(['Start Date1', 'End Date1'], inplace = True, axis = 1)

#############################################################################

masterERPPostingProper = masterERPPosting[masterERPPosting['Flat Posting IPR'] == '0']
masterERPPostingProper = masterERPPostingProper[~(masterERPPostingProper['Posted Date'].isna())]

masterERPPostingProper['Current Posting Counter'] = masterERPPostingProper['Posted Date']
masterERPPostingProper['First Posting'] = masterERPPostingProper['Posted Date']
masterERPPostingProper['Latest Posting'] = masterERPPostingProper['Posted Date']

masterERPPostingProperSummary = masterERPPostingProper.groupby(['Kibana ID','Bill Types','Bill Frequency']).agg({'Current Posting Counter':'count','First Posting':'min','Latest Posting':'max'}).reset_index()
masterERPPostingProper = pd.merge(masterERPPostingProper, masterERPPostingProperSummary, on = ['Kibana ID','Bill Types','Bill Frequency'], how = 'left')
# masterERPPostingProperSummary = masterERPPostingProper.groupby(['Kibana ID','Bill Types','Bill Frequency']).agg({'Posted Date': ['count','min', 'max']}).reset_index()

# masterERPPostingProper = pd.merge(masterERPPostingProper, masterERPPostingProperSummary, on = ['Kibana ID','Bill Types','Bill Frequency'], how = 'left')

masterERPPostingProper = masterERPPostingProper[~(masterERPPostingProper[('Posted Date')].isna())]

def diff_month(row):
  strt_dt = row[('Posted Date')]
  end_dt = pd.to_datetime("today")
  return (end_dt.year - strt_dt.year) * 12 + end_dt.month - strt_dt.month + int(round((end_dt.day - strt_dt.day)/28))

masterERPPostingProper['Current Counter']= masterERPPostingProper.apply(diff_month,axis=1)
masterERPPostingProper['Current Counter'] = masterERPPostingProper['Current Counter']/masterERPPostingProper['Bill Frequency']

masterERPPostingProper.rename({'Current Posting Counter_y': 'Current Posting Counter', 'Current Counter':'Current Billing Counter','First Posting_y':'First Posting','Latest Posting_y':'Latest Posting'}, inplace = True, axis = 1)

masterERPPostingProper = masterERPPostingProper[['Kibana ID', 'Posted Date', 'Deletion Status', 'Bill Types', 'Bill Frequency', 'combined IPR', 'Flat Count',
                                                 'Total Posted Amount in K','Current Billing Counter','Current Posting Counter', 'First Posting', 'Latest Posting','Postingperc','TransactionPerc']]

masterERPPostingProper['Posted Month'] = pd.to_datetime(masterERPPostingProper['Posted Date']).dt.month

masterERPPostingProper = masterERPPostingProper.sort_values(['Total Posted Amount in K']).groupby(['Kibana ID','Posted Month']).tail(1)
masterERPPostingProper = masterERPPostingProper.sort_values(['Posted Date']).groupby(['Kibana ID']).tail(1)
masterERPPostingProper.drop(['Posted Month'],inplace = True, axis = 1)
masterERPPostingProper.drop_duplicates(inplace = True)

<ipython-input-148-197d8be49f70>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  masterERPPostingProper['Posted Month'] = pd.to_datetime(masterERPPostingProper['Posted Date']).dt.month


In [ ]:
masterERPPostingProper[masterERPPostingProper['Kibana ID']=='8a96bacc92585f7d01925be0a69d012f']

,Kibana ID,Posted Date,Deletion Status,Bill Types,Bill Frequency,combined IPR,Flat Count,Total Posted Amount in K,Current Billing Counter,Current Posting Counter,First Posting,Latest Posting,Postingperc,TransactionPerc
6108,8a96bacc92585f7d01925be0a69d012f,2025-03-01,Not Deleted,MaintainanceAUTOTAG,1.0,0||0||0,168.0,161.0,0.0,5,2024-11-01,2025-03-01,100.0,132.738095


In [ ]:
closures['Sold Flats'] = closures[['ERP Sold Flats','VMS Sold Flats as per Occupancy']].max(axis=1)

In [ ]:
closures = pd.merge(closures, society_creation_date, on = ['Kibana ID'], how = 'left')
closures = pd.merge(closures, Download, on = ['Kibana ID'], how = 'left')
closures = pd.merge(closures, account, on = ['Kibana ID'], how = 'left')
closures = pd.merge(closures, masterERPPostingProper, on = ['Kibana ID'], how = 'left')
closures = pd.merge(closures, masterTransaction, on = ['Kibana ID'], how = 'left')
#closures = pd.merge(closures, vendor_30days, on = ['Kibana ID'], how = 'left')
#closures = pd.merge(closures, receipt_entry, on = ['Kibana ID'], how = 'left')
# closures = pd.merge(closures, IPR_condition, on = ['Kibana ID'], how = 'left')
closures.loc[:,'Download Perc'] = closures['Download Count']/closures['Sold Flats']*100

closures.drop_duplicates(inplace=True)

In [ ]:
closures[closures['Kibana ID']=='8a96bacc92585f7d01925be0a69d012f'][['Closure Number', 'Kibana ID','combined IPR']]

,Closure Number,Kibana ID,combined IPR
8141,1038094,8a96bacc92585f7d01925be0a69d012f,0||0||0


#**🎲Multipliers**#

In [ ]:
############# Trail Factor Reduction Multiplier #############

closures['Closure Date'] = pd.to_datetime(closures['Closure Date'])

def trialfactor(row):
    if row['Sales Product'] in ['VMS only', 'Both VMS + ERP'] and pd.Timestamp(row['Closure Date']) > pd.to_datetime("30 Nov 2023"):
        if row['Competition Multiplier (PIA) - VMS'] > 1:
            return 1
        else:
            if row['VMSTrialPeriod'] != "" and row['VMSTrialPeriod'] in ["4 Year (Limited period)", "5 Year (Limited period)", "6 Year (Limited period)", "7 Year (Limited period)", "8 Year (Limited period)", "9 Year (Limited period)", "10 Year (Limited period)", "Lifetime Free offer"]:
                return 0.8
            if row['VMSTrialPeriod'] != "" and row['VMSTrialPeriod'] in ["2 Year (Limited period)", "3 Year (Limited period)"]:
                return 0.9
            else:
                return 1
    else:
      return 1

closures['VMSTrialReductionFactor'] = closures.apply(trialfactor, axis=1)


In [ ]:
######### Closure month cycle #########
closures['year'] = np.where((closures['monthDays']>=26) & (closures['Closure Month']==12) & (closures['year']==2023),closures['year']+1,closures['year'])
closures['year'] = np.where((closures['monthDays']>=27) & (closures['Closure Month']==12) & (closures['year']==2024),closures['year']+1,closures['year'])

def monthcycle(row):
  if row['Closure Date']>= pd.to_datetime("26 Dec 2023") and row['Closure Date']<= pd.to_datetime("26 Jan 2024"):
    return 1
  if row['Closure Date']>= pd.to_datetime("27 Jan 2024") and row['Closure Date']<= pd.to_datetime("26 Feb 2024"):
    return 2
  if row['Closure Date']>= pd.to_datetime("27 Feb 2024") and row['Closure Date']<= pd.to_datetime("26 Mar 2024"):
    return 3
  if row['Closure Date']>= pd.to_datetime("27 Mar 2024") and row['Closure Date']<= pd.to_datetime("28 Apr 2024"):
    return 4
  if row['Closure Date']>= pd.to_datetime("29 Apr 2024") and row['Closure Date']<= pd.to_datetime("26 May 2024"):
    return 5
  if row['Closure Date']>= pd.to_datetime("27 May 2024") and row['Closure Date']<= pd.to_datetime("26 Jun 2024"):
    return 6
  if row['Closure Date']>= pd.to_datetime("27 Jun 2024") and row['Closure Date']<= pd.to_datetime("26 Jul 2024"):
    return 7
  if row['Closure Date']>= pd.to_datetime("27 Jul 2024") and row['Closure Date']<= pd.to_datetime("26 Aug 2024"):
    return 8
  if row['Closure Date']>= pd.to_datetime("27 Aug 2024") and row['Closure Date']<= pd.to_datetime("26 Sep 2024"):
    return 9
  if row['Closure Date']>= pd.to_datetime("27 Sep 2024") and row['Closure Date']<= pd.to_datetime("26 Oct 2024"):
    return 10
  if row['Closure Date']>= pd.to_datetime("27 Oct 2024") and row['Closure Date']<= pd.to_datetime("26 Nov 2024"):
    return 11
  if row['Closure Date']>= pd.to_datetime("27 Nov 2024") and row['Closure Date']<= pd.to_datetime("26 Dec 2024"):
    return 12
  if row['Closure Date']>= pd.to_datetime("27 Dec 2024") and row['Closure Date']<= pd.to_datetime("26 Jan 2025"):
    return 1
  if row['Closure Date']>= pd.to_datetime("27 Jan 2025") and row['Closure Date']<= pd.to_datetime("26 Feb 2025"):
    return 2
  if row['Closure Date']>= pd.to_datetime("27 Feb 2025") and row['Closure Date']<= pd.to_datetime("26 Mar 2025"):
    return 3
  else:
    return row['Closure Month']

closures['Closure Month'] = closures.apply(monthcycle, axis = 1)


In [ ]:

######### Date Multiplier #########

closures['RP_D4'] = closures['RP_D4'].replace('Not Eligible for signing', '', regex=True)
closures[['ERPHOFilledDate','VMSHOFilledDate']] = closures[['ERPHOFilledDate','VMSHOFilledDate']].apply(changeDateFormat)

def Date_Multiplier(row):
    closure_filled_date = pd.Timestamp(row['Closure Form Filled Date'])

    if pd.to_datetime("26 Dec 2023") <= closure_filled_date <= pd.to_datetime("5 Jan 2024") and ((pd.Timestamp(row['VMSHOFilledDate']) <= pd.to_datetime("10 Jan 2024")) or (pd.Timestamp(row['ERPHOFilledDate']) <= pd.to_datetime("10 Jan 2024"))):
        try:
            rp_d4 = float(row['RP_D4'])
            if rp_d4 <= 200000:
                return 1.5
            else:
                return 1.25
        except ValueError:
            return 1
    elif pd.to_datetime("27 Jan 2024") <= closure_filled_date <= pd.to_datetime("4 Feb 2024") and ((pd.Timestamp(row['VMSHOFilledDate']) <= pd.to_datetime("10 Feb 2024")) or (pd.Timestamp(row['ERPHOFilledDate']) <= pd.to_datetime("10 Feb 2024"))):
        # Date range: 27th Jan 2024 to 4th Feb 2024
        try:
            rp_d4 = float(row['RP_D4'])
            if rp_d4 < 200000:
                return 1.5
            else:
                return 1.25
        except ValueError:
            return 1
    elif pd.to_datetime("14 Mar 2024") <= closure_filled_date <= pd.to_datetime("17 Mar 2024") and ((pd.Timestamp(row['VMSHOFilledDate']) <= pd.to_datetime("24 Mar 2024")) or (pd.Timestamp(row['ERPHOFilledDate']) <= pd.to_datetime("24 Mar 2024"))):
        # Date range: 27th Jan 2024 to 4th Feb 2024
        try:
            rp_d4 = float(row['RP_D4'])
            if rp_d4 < 200000:
                return 1.5
            else:
                return 1.25
        except ValueError:
            return 1
    elif pd.to_datetime("9 Apr 2024") <= closure_filled_date <= pd.to_datetime("14 Apr 2024") and ((pd.Timestamp(row['VMSHOFilledDate']) <= pd.to_datetime("22 Apr 2024")) or (pd.Timestamp(row['ERPHOFilledDate']) <= pd.to_datetime("22 Apr 2024"))):
        # Date range: 27th Jan 2024 to 4th Feb 2024
        try:
            rp_d4 = float(row['RP_D4'])
            if rp_d4 < 200000:
                return 1.5
            else:
                return 1.25
        except ValueError:
            return 1
    elif pd.to_datetime("10 May 2024") <= closure_filled_date <= pd.to_datetime("13 May 2024") and ((pd.Timestamp(row['VMSHOFilledDate']) <= pd.to_datetime("19 May 2024")) or (pd.Timestamp(row['ERPHOFilledDate']) <= pd.to_datetime("19 May 2024"))):
        try:
            rp_d4 = float(row['RP_D4'])
            if rp_d4 < 200000:
                return 1.5
            else:
                return 1.25
        except ValueError:
            return 1
    elif pd.to_datetime("5 Sep 2024") <= closure_filled_date <= pd.to_datetime("10 Sep 2024") and ((pd.Timestamp(row['VMSHOFilledDate']) <= pd.to_datetime("18 Sep 2024")) or (pd.Timestamp(row['ERPHOFilledDate']) <= pd.to_datetime("18 Sep 2024"))):
        try:
            rp_d4 = float(row['RP_D4'])
            if rp_d4 < 200000:
                return 1.5
            else:
                return 1.25
        except ValueError:
            return 1
    elif pd.to_datetime("12 Oct 2024") <= closure_filled_date <= pd.to_datetime("14 Oct 2024") and ((pd.Timestamp(row['VMSHOFilledDate']) <= pd.to_datetime("20 Oct 2024")) or (pd.Timestamp(row['ERPHOFilledDate']) <= pd.to_datetime("20 Oct 2024"))):
        try:
            rp_d4 = float(row['RP_D4'])
            if rp_d4 < 200000:
                return 1.5
            else:
                return 1.25
        except ValueError:
            return 1
    elif pd.to_datetime("2 Nov 2024") <= closure_filled_date <= pd.to_datetime("4 Nov 2024") and ((pd.Timestamp(row['VMSHOFilledDate']) <= pd.to_datetime("10 Nov 2024")) or (pd.Timestamp(row['ERPHOFilledDate']) <= pd.to_datetime("10 Nov 2024"))):
        try:
            rp_d4 = float(row['RP_D4'])
            if rp_d4 < 200000:
                return 1.5
            else:
                return 1.25
        except ValueError:
            return 1
    elif pd.to_datetime("10 Jan 2025") <= closure_filled_date <= pd.to_datetime("12 Jan 2025") and ((pd.Timestamp(row['VMSHOFilledDate']) <= pd.to_datetime("18 Jan 2025")) or (pd.Timestamp(row['ERPHOFilledDate']) <= pd.to_datetime("18 Jan 2025"))):
        try:
            rp_d4 = float(row['RP_D4'])
            if rp_d4 < 200000:
                return 1.5
            else:
                return 1.25
        except ValueError:
            return 1
    elif pd.to_datetime("14 Feb 2025") <= closure_filled_date <= pd.to_datetime("16 Feb 2025") and ((pd.Timestamp(row['VMSHOFilledDate']) <= pd.to_datetime("21 Feb 2025")) or (pd.Timestamp(row['ERPHOFilledDate']) <= pd.to_datetime("21 Feb 2025"))):
        try:
            rp_d4 = float(row['RP_D4'])
            if rp_d4 < 200000:
                return 1.5
            else:
                return 1.25
        except ValueError:
            return 1
    else:
        return 1
closures['Date_Multiplier'] = closures.apply(Date_Multiplier, axis=1)

closures.loc[closures['Closure Number'].isin([1035163,1035193,1034237,1035025,1035165, 1035056,1036647,1036648,1036649,1036603,1034008,1020107,1025598,1039460]),'Date_Multiplier'] = 1.5
closures.loc[closures['Closure Number'].isin([1036725,1036717,1036785,1036852,1036821,1036529]),'Date_Multiplier'] = 1

closures['Date_Multiplier'].value_counts()


,count
Date_Multiplier,
1.00,3938
1.50,504
1.25,20


In [ ]:

def Agreement_Multiplier(row):
  if row['Agreement Type'] in ['1 Pager Agreement with Penalty Clause'] and row['Agreement Validity'] == 1:
    return 1.2
  else:
    return 1

closures['Agreement_Multiplier'] = closures.apply(Agreement_Multiplier, axis=1)

closures['Agreement_Multiplier'].value_counts()

,count
Agreement_Multiplier,
1.0,4306
1.2,156


In [ ]:
########## Monetization Priority Multiplier ###########

closures['Monetization Multiplier'] = 1
closures.loc[closures['Closure Number'].isin([1034568,1034938,1035193,1035163,1035046,1035494,1035213,1035259,1035109,1035010,1034558,1034428,1034391,1035206,1035046,1035681,1035842,
                                              1035940,1036005,1036144,1036734,1036731,1036739,1036970,1037306,1036990,1037217,1037706,1037690,1037401,1037363,1037995,1038016,1037855,
                                              1037851,1037858,1038105,1037724,1037871,1037837,1038031,1037850,1037699,1038158,1038130,1038129,1038212,1038208,1038245,1038522,
                                              1038551,1038563,1038493,1038542,1037864,1038526,1038946,1038787,1038786,1038784,1038782,1038785,1038668, 1039007,1039037,1039309,1039113,1038961]),'Monetization Multiplier'] = 1.5

closures.loc[closures['Closure Number'].isin([1037385, 1037330, 1037470,1037830,1037990,1038157,1038251,1038194,1038185,1038234,1038390,1038636,1038525,1038492,1038630,1038820,1038812,1039048,1038800,1038476,1039241,1039233,1039176,1039023,1039024]),'Monetization Multiplier'] = 1.75

############### Estimated Device Cost Calculation ################
selected_rows = closures['Closure Number'].isin([1037385, 1037429, 1037330, 1037706, 1037690, 1037830,1038016,1037851,1037995,1037858,1038105,1037990,1038031,1037699,1038251,1038212,
                                                 1038194,1038234,1038476,1038522,1038390,1038493,1038525,1038492,1038129,1038130,1038946,1038668,1039048,1038800,1039037,1039007,1039309,1039241,1039233,1039176,1039113,1039023,1039024])

############### Remove VMS trail factor multiplier ################
TrialReductionFactor_List = [1039037, 1039007,1039048]
closures.loc[closures['Closure Number'].isin(TrialReductionFactor_List), 'VMSTrialReductionFactor'] = 1

fc_limits = pd.DataFrame({
    'FC Lower Limit': [0, 251, 501, 751, 1001, 1251, 1501, 2001],
    'FC Upper Limit': [250, 500, 750, 1000, 1250, 1500, 2000, 10000],
    'Devices': [1, 2, 3, 4, 5, 6, 7, 8]
})

def find_device_count(fc_number):
    slab = fc_limits[(fc_limits['FC Lower Limit'] <= fc_number) & (fc_limits['FC Upper Limit'] >= fc_number)]

    return slab['Devices'].values[0] if not slab.empty else 0

# Apply the function to each FC number in the data
closures['Estimated Device Cost'] = closures['Sold Flats'].apply(find_device_count)*9900

<ipython-input-157-a49119ac3ee6>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  closures.loc[closures['Closure Number'].isin([1034568,1034938,1035193,1035163,1035046,1035494,1035213,1035259,1035109,1035010,1034558,1034428,1034391,1035206,1035046,1035681,1035842,


In [ ]:
# VMSTrialReductionFactor Check
closures[closures['Closure Number'].isin([1038800,1039037,1039007,1039048])][['Closure Number','VMSTrialReductionFactor']]

,Closure Number,VMSTrialReductionFactor
10695,1038800,1.0
10998,1039007,1.0
11049,1039048,1.0
11086,1039037,1.0


#**🎲D1 Eligibility Checker**#

In [ ]:
def D1ERP_Eligible(row):
    if pd.isna(row['combined IPR']):
      if row['ERP Onb Status'] == 'ON-BOARDED' and row['HO Product'] == 'ERP only':
        return "Yes"
      else:
        return "No"
    if row['HO Product'] in ['ERP only', 'Both VMS + ERP']:
        ipr_values = row['combined IPR'].split('||') if isinstance(row['combined IPR'], str) else ['0', '0', '0']

        if ipr_values == ['0', '0', '0']:
            return "Yes"
        else:
          return "No"

    return "No"

closures['RP_D1_ERPEligible'] = closures.apply(D1ERP_Eligible, axis=1)
closures.loc[closures['HO Product'] == 'VMS only', 'RP_D1_ERPEligible'] = ""

closures.loc[(closures['HO Product'] == 'ERP only') & (closures['Onb Status'] == 'ON-BOARDED'), 'RP_D1_ERPEligible'] = "Yes"

In [ ]:
#Added by Prashant for D1 Updation.
def downloadslab(row):
    DownloadMultiplier = ""

    # Check if 'Download Perc' is NaN
    if pd.isna(row['Download Perc']):
        DownloadMultiplier = "No"
    else:
        # If 'Download Perc' is less than 30
        if row['Download Perc'] < 30:
            DownloadMultiplier = "No"
        # If 'Download Perc' is 30 or greater
        elif row['Download Perc'] >= 30:
            DownloadMultiplier = "Yes"

    return DownloadMultiplier

# Apply the function
closures['RP_D1_VMSEligible'] = closures.apply(downloadslab, axis=1)
closures.loc[closures['HO Product'] == 'ERP only', 'RP_D1_VMSEligible'] = ""

closures.loc[(closures['HO Product'] == 'VMS only') & (closures['Onb Status'] == 'ON-BOARDED'), 'RP_D1_VMSEligible'] = "Yes"

In [ ]:
  closures[closures['Closure Number'].isin([1037422, 1037449])][['Closure Number','HO Product','Download Perc','combined IPR','RP D1','RP_D1_VMSEligible','RP_D1_ERPEligible','Onb Status']]

,Closure Number,HO Product,Download Perc,combined IPR,RP D1,RP_D1_VMSEligible,RP_D1_ERPEligible,Onb Status
6793,1037422,Both VMS + ERP,91.818182,0||0||0,54865.0739,Yes,Yes,ON-BOARDED
6856,1037449,Both VMS + ERP,60.759494,0||0||0,36170.6479,Yes,Yes,ON-BOARDED


In [ ]:
closures[['RP_HO', 'RP D1', 'RP_D4','Society RP as per Sales Product','Society RP as per HO Product','Welfare Amount','New device cost', 'Refurbished device cost']] = closures[['RP_HO', 'RP D1', 'RP_D4','Society RP as per Sales Product','Society RP as per HO Product','Welfare Amount','New device cost', 'Refurbished device cost']].apply(pd.to_numeric, errors='coerce').fillna(0)
closures[['RP_HO', 'RP D1', 'RP_D4','Society RP as per Sales Product','Society RP as per HO Product','Welfare Amount','New device cost', 'Refurbished device cost']] = closures[['RP_HO', 'RP D1', 'RP_D4','Society RP as per Sales Product','Society RP as per HO Product','Welfare Amount','New device cost', 'Refurbished device cost']].astype(int)

In [ ]:
#Added by Prashant for D1 Updation.
conditions = [
    (closures['HO Product'] == 'Both VMS + ERP') & (closures['RP_D1_ERPEligible'] == 'No') & (closures['RP_D1_VMSEligible'] == 'No') ,
    (closures['HO Product'] == 'Both VMS + ERP') & (closures['RP_D1_ERPEligible'] == 'No') & (closures['RP_D1_VMSEligible'] == 'Yes') ,
    (closures['HO Product'] == 'Both VMS + ERP') & (closures['RP_D1_VMSEligible'] == 'No') & (closures['RP_D1_ERPEligible'] == 'Yes') ,
    (closures['HO Product'] == 'ERP only') & (closures['RP_D1_ERPEligible'] == 'No'),
    (closures['HO Product'] == 'VMS only') & (closures['RP_D1_VMSEligible'] == 'No')
]

# Define the corresponding outputs
outputs = [
    "Not Eligible",  # Both VMS + ERP and both eligible fields are 'No'
    "VMS only",      # Both VMS + ERP, but ERP is 'No'
    "ERP only",      # Both VMS + ERP, but VMS is 'No'
    "Not Eligible",  # ERP only and ERP eligible is 'No'
    "Not Eligible"   # VMS only and VMS eligible is 'No'
]

# Use np.select to apply the conditions
closures['D1 Product'] = np.select(conditions, outputs, default=closures['HO Product'])

closures.loc[:,'Competition Multiplier (PIA) - VMS'] = pd.to_numeric(closures['Competition Multiplier (PIA) - VMS'])
closures.loc[:,'Competition Multiplier (PIA) - ERP'] = pd.to_numeric(closures['Competition Multiplier (PIA) - ERP'])

# For ERP Onboarded Meter Integration sites & Both VMS_ERP sites D1:
meter = (closures['Closure Number'].isin([1037422, 1037449, 1037384, 1037650, 1038061, 1036740, 1035983, 1036811,1038195, 1038187, 1038173, 1038175, 1038315, 1038309, 1038165, 1038136, 1038094]))
closures.loc[meter, 'RP_D1_ERPEligible'] = 'Yes'
closures.loc[meter, 'D1 Product'] = closures['HO Product']

# ERP_Onb = (closures['HO Product'] == 'ERP only') & (closures['ERP Onb Status'] == 'ON-BOARDED')
# closures.loc[ERP_Onb, 'RP_D1_ERPEligible'] = 'Yes'
# closures.loc[ERP_Onb, 'D1 Product'] = closures['HO Product']

VMS_ERP = (closures['HO Product'] == 'Both VMS + ERP') & (closures['ERP Onb Status'] == 'ON-BOARDED') & (closures['RP_D1_VMSEligible'] == 'Yes')
closures.loc[VMS_ERP, 'RP_D1_ERPEligible'] = 'Yes'
closures.loc[VMS_ERP, 'D1 Product'] = closures['HO Product']

closures.loc[((closures['HO Product'] == 'VMS only') & (closures['D1 Product'] == 'VMS only') & (closures['RP_D1_VMSEligible'] == 'Yes') & (closures['Agreement Validity']==1 )),'RP_D1'] = (closures['RP D1'])
closures.loc[((closures['HO Product'] == 'Both VMS + ERP') & (closures['D1 Product'] == 'VMS only') & (closures['RP_D1_VMSEligible'] == 'Yes')& (closures['Agreement Validity']==1)),'RP_D1'] = (closures['RP D1'])*0.5
closures.loc[((closures['HO Product'] == 'Both VMS + ERP') & (closures['D1 Product'] == 'ERP only') & (closures['RP_D1_ERPEligible'] == 'Yes')& (closures['Agreement Validity']==1)),'RP_D1'] = (closures['RP D1'])*0.5
closures.loc[((closures['HO Product'] == 'ERP only') & (closures['D1 Product'] == 'ERP only') & (closures['RP_D1_ERPEligible'] == 'Yes') & (closures['Agreement Validity']==1)),'RP_D1'] = closures['RP D1']

closures.loc[((closures['D1 Product'] == 'Both VMS + ERP') & (closures['RP_D1_ERPEligible'] == 'Yes') & (closures['RP_D1_VMSEligible'] == 'Yes') & (closures['Agreement Validity']==1)),'RP_D1'] = closures['RP D1']

closures.loc[((closures['Onb Status'] == 'ON-BOARDED') & (closures['Property Type'] == 'Builder') & (closures['Agreement Validity']==1)),'RP_D1'] = closures['RP D1']

closures.loc[closures['Closure Number'].isin([1034386,1034134,1034318,1034670,1028003,1029390,1020338,1034868,1034909,1034835,1018732,1035432,1034711,1035410,1034981,1034951,1035228,1035192,1035171,
                                              1035025,1035145,1035200,1035104,1034987,1035037,1035885,1035146,1035171,1036052,1035991,1035959,1035920,1036407,1036565,1036946,
                                              1036496,1037298,1037078,1037680,1036990,1037007]),'RP_D1'] = closures['RP D1']

In [ ]:
############### RP_D1 for closure where Net society RP is > 1,50,000 ###############
'''
valid_date = pd.to_datetime("26 Oct 2024")
rp_criteria = closures['Society RP as per HO Product'] > 150000
agreement_valid = closures['Agreement Validity'] == 1
closure_date_criteria = closures['Closure Date'] > valid_date

# Condition 1: VMS only
closures.loc[
    (closures['HO Product'] == 'VMS only') &
    (closures['D1 Product'] == 'VMS only') &
    (closures['RP_D1_VMSEligible'] == 'Yes') &
    agreement_valid & closure_date_criteria &
    (closures['VMS Onb Status'] == 'ON-BOARDED') &
    rp_criteria,
    'RP_D1'
] = closures['RP D1']

# Condition 2: Both VMS + ERP and D1 Product VMS only
closures.loc[
    (closures['HO Product'] == 'Both VMS + ERP') &
    (closures['D1 Product'] == 'VMS only') &
    (closures['RP_D1_VMSEligible'] == 'Yes') &
    agreement_valid & closure_date_criteria &
    (closures['VMS Onb Status'] == 'ON-BOARDED') &
    rp_criteria,
    'RP_D1'
] = closures['RP D1'] * 0.5

# Condition 3: Both VMS + ERP and D1 Product ERP only
closures.loc[
    (closures['HO Product'] == 'Both VMS + ERP') &
    (closures['D1 Product'] == 'ERP only') &
    (closures['RP_D1_ERPEligible'] == 'Yes') &
    agreement_valid & closure_date_criteria &
    (closures['ERP Onb Status'] == 'ON-BOARDED') &
    rp_criteria,
    'RP_D1'
] = closures['RP D1'] * 0.4

# Condition 4: ERP only
closures.loc[
    (closures['HO Product'] == 'ERP only') &
    (closures['D1 Product'] == 'ERP only') &
    (closures['RP_D1_ERPEligible'] == 'Yes') &
    agreement_valid & closure_date_criteria &
    (closures['ERP Onb Status'] == 'ON-BOARDED') &
    rp_criteria,
    'RP_D1'
] = closures['RP D1']

# Condition 5: Both VMS + ERP for both HO Product and D1 Product
closures.loc[
    (closures['HO Product'] == 'Both VMS + ERP') &
    (closures['D1 Product'] == 'Both VMS + ERP') &
    (closures['RP_D1_VMSEligible'] == 'Yes') &
    (closures['RP_D1_ERPEligible'] == 'Yes') &
    agreement_valid & closure_date_criteria &
    (closures['VMS Onb Status'] == 'ON-BOARDED') &
    (closures['ERP Onb Status'] == 'ON-BOARDED') &
    rp_criteria,
    'RP_D1'
] = closures['RP D1']
'''

'\nvalid_date = pd.to_datetime("26 Oct 2024")\nrp_criteria = closures[\'Society RP as per HO Product\'] > 150000\nagreement_valid = closures[\'Agreement Validity\'] == 1\nclosure_date_criteria = closures[\'Closure Date\'] > valid_date\n\n# Condition 1: VMS only\nclosures.loc[\n    (closures[\'HO Product\'] == \'VMS only\') &\n    (closures[\'D1 Product\'] == \'VMS only\') &\n    (closures[\'RP_D1_VMSEligible\'] == \'Yes\') &\n    agreement_valid & closure_date_criteria &\n    (closures[\'VMS Onb Status\'] == \'ON-BOARDED\') &\n    rp_criteria,\n    \'RP_D1\'\n] = closures[\'RP D1\']\n\n# Condition 2: Both VMS + ERP and D1 Product VMS only\nclosures.loc[\n    (closures[\'HO Product\'] == \'Both VMS + ERP\') &\n    (closures[\'D1 Product\'] == \'VMS only\') &\n    (closures[\'RP_D1_VMSEligible\'] == \'Yes\') &\n    agreement_valid & closure_date_criteria &\n    (closures[\'VMS Onb Status\'] == \'ON-BOARDED\') &\n    rp_criteria,\n    \'RP_D1\'\n] = closures[\'RP D1\'] * 0.5\n\n# Conditio

In [ ]:


closures.loc[selected_rows, 'RP_D4'] = closures.loc[selected_rows, 'RP_D4'] + closures.loc[selected_rows, 'Welfare Amount']
closures.loc[selected_rows, 'RP_HO'] = closures.loc[selected_rows, 'RP_HO'] + closures.loc[selected_rows, 'Welfare Amount']
closures.loc[selected_rows, 'RP_D1'] = closures.loc[selected_rows, 'RP_D1'] + closures.loc[selected_rows, 'Welfare Amount']

closures.loc[selected_rows, 'RP_D4'] = closures.loc[selected_rows, 'RP_D4'] + closures.loc[selected_rows, 'New device cost']
closures.loc[selected_rows, 'RP_HO'] = closures.loc[selected_rows, 'RP_HO'] + closures.loc[selected_rows, 'New device cost']
closures.loc[selected_rows, 'RP_D1'] = closures.loc[selected_rows, 'RP_D1'] + closures.loc[selected_rows, 'New device cost']

closures.loc[selected_rows, 'RP_D4'] = closures.loc[selected_rows, 'RP_D4'] + closures.loc[selected_rows, 'Refurbished device cost']
closures.loc[selected_rows, 'RP_HO'] = closures.loc[selected_rows, 'RP_HO'] + closures.loc[selected_rows, 'Refurbished device cost']
closures.loc[selected_rows, 'RP_D1'] = closures.loc[selected_rows, 'RP_D1'] + closures.loc[selected_rows, 'Refurbished device cost']

closures.loc[selected_rows, 'RP_D4'] = closures.loc[selected_rows, 'RP_D4'] - closures.loc[selected_rows, 'Estimated Device Cost']
closures.loc[selected_rows, 'RP_HO'] = closures.loc[selected_rows, 'RP_HO'] - closures.loc[selected_rows, 'Estimated Device Cost']
closures.loc[selected_rows, 'RP_D1'] = closures.loc[selected_rows, 'RP_D1'] - closures.loc[selected_rows, 'Estimated Device Cost']

# Convert selected_rows['Closure Number'] to a list or Series, if it's a DataFrame
closures.loc[selected_rows, 'New device cost'] = closures.loc[selected_rows, 'Estimated Device Cost']

###########################################################################

#Applying Trail Period Reduction & Date Multiplier
closures['RP_D4'] = closures['RP_D4'] * closures['Agreement_Multiplier'] * closures['VMSTrialReductionFactor'] * closures['Date_Multiplier'] * closures['Monetization Multiplier'] * closures[['Competition Multiplier (PIA) - VMS', 'Competition Multiplier (PIA) - ERP']].max(axis=1)
closures['RP_HO'] = closures['RP_HO'] * closures['Agreement_Multiplier'] * closures['VMSTrialReductionFactor'] * closures['Date_Multiplier'] * closures['Monetization Multiplier'] * closures[['Competition Multiplier (PIA) - VMS', 'Competition Multiplier (PIA) - ERP']].max(axis=1)
closures['RP_D1'] = closures['RP_D1'] * closures['Agreement_Multiplier'] * closures['VMSTrialReductionFactor'] * closures['Date_Multiplier'] * closures['Monetization Multiplier'] * closures[['Competition Multiplier (PIA) - VMS', 'Competition Multiplier (PIA) - ERP']].max(axis=1)


closures.loc[closures['Society Type']=='Commercial','RP_D4'] = 0
closures.loc[closures['Society Type']=='Commercial','RP_HO'] = 0
closures.loc[closures['Society Type']=='Commercial','RP_D1'] = 0
closures.loc[closures['Society Type']=='Commercial','Gross RP Without Offset'] = 0
closures.loc[closures['Society Type']=='Commercial','Gross RP - PIA_Validated RP Component'] = 0
closures.loc[closures['Society Type']=='Commercial','Gross RP - PIA_Only Valid'] = 0
closures.loc[closures['Society Type']=='Commercial','Gross RP - PIA as per D1 Product'] = 0
closures.loc[closures['Society Type']=='Commercial','Gross RP - PIA_Validated RP Component Onboarded'] = 0

closures.loc[closures['Closure City'].isin(['Indore','Lucknow','Kochi','Visakapatanam','Coimbatore','Raipur','Surat','Vadodara','Chandigarh','Goa','Sonipat','Manesar','Merut','Other City (Non-operational Cities)']),'RP_D4'] = 0
closures.loc[closures['Closure City'].isin(['Indore','Lucknow','Kochi','Visakapatanam','Coimbatore','Raipur','Surat','Vadodara','Chandigarh','Goa','Sonipat','Manesar','Merut','Other City (Non-operational Cities)']),'RP_HO'] = 0
closures.loc[closures['Closure City'].isin(['Indore','Lucknow','Kochi','Visakapatanam','Coimbatore','Raipur','Surat','Vadodara','Chandigarh','Goa','Sonipat','Manesar','Merut','Other City (Non-operational Cities)']),'RP_D1'] = 0
closures.loc[closures['Closure City'].isin(['Indore','Lucknow','Kochi','Visakapatanam','Coimbatore','Raipur','Surat','Vadodara','Chandigarh','Goa','Sonipat','Manesar','Merut','Other City (Non-operational Cities)']),'Gross RP Without Offset'] = 0
closures.loc[closures['Closure City'].isin(['Indore','Lucknow','Kochi','Visakapatanam','Coimbatore','Raipur','Surat','Vadodara','Chandigarh','Goa','Sonipat','Manesar','Merut','Other City (Non-operational Cities)']),'Gross RP - PIA_Validated RP Component'] = 0
closures.loc[closures['Closure City'].isin(['Indore','Lucknow','Kochi','Visakapatanam','Coimbatore','Raipur','Surat','Vadodara','Chandigarh','Goa','Sonipat','Manesar','Merut','Other City (Non-operational Cities)']),'Gross RP - PIA_Only Valid'] = 0
closures.loc[closures['Closure City'].isin(['Indore','Lucknow','Kochi','Visakapatanam','Coimbatore','Raipur','Surat','Vadodara','Chandigarh','Goa','Sonipat','Manesar','Merut','Other City (Non-operational Cities)']),'Gross RP - PIA as per D1 Product'] = 0
closures.loc[closures['Closure City'].isin(['Indore','Lucknow','Kochi','Visakapatanam','Coimbatore','Raipur','Surat','Vadodara','Chandigarh','Goa','Sonipat','Manesar','Merut','Other City (Non-operational Cities)']),'Gross RP - PIA_Validated RP Component Onboarded'] = 0


closures.loc[closures['Closure Done By (Secondary Email)'] == "", 'PrimaryRPsHO'] = closures['RP_HO']
closures.loc[closures['Closure Done By (Secondary Email)'] == "", 'SecondaryRPsHO'] = 0
closures.loc[closures['Closure Done By (Secondary Email)'] != "", 'PrimaryRPsHO'] = closures['RP_HO']/2
closures.loc[closures['Closure Done By (Secondary Email)'] != "", 'SecondaryRPsHO'] = closures['RP_HO']/2

closures.loc[closures['Closure Done By (Secondary Email)'] == "", 'PrimaryRPsD1'] = closures['RP_D1']
closures.loc[closures['Closure Done By (Secondary Email)'] == "", 'SecondaryRPsD1'] = 0
closures.loc[closures['Closure Done By (Secondary Email)'] != "", 'PrimaryRPsD1'] = closures['RP_D1']/2
closures.loc[closures['Closure Done By (Secondary Email)'] != "", 'SecondaryRPsD1'] = closures['RP_D1']/2

# closures['Sold Flats'] = closures[['ERP Sold Flats','VMS Sold Flats as per Occupancy']].max(axis=1)

closuresDuplicateClosureNumber = closures[closures['Closure Number'].duplicated()]['Closure Number'].drop_duplicates()
closures.loc[closures['Closure Number'].isin(closuresDuplicateClosureNumber),'DuplicateClosureNumber'] = True

In [ ]:
closures[closures['Closure Number']==1038800][['Closure Number','Closure Date', 'Sales Product','HO Product','D1 Product','ERP Onb Status','VMS Onb Status','combined IPR','Download Perc','RP_D1_VMSEligible','RP_D1_ERPEligible','Society RP as per Sales Product','RP_D4','RP_HO','RP_D1']].head(1)

,Closure Number,Closure Date,Sales Product,HO Product,D1 Product,ERP Onb Status,VMS Onb Status,combined IPR,Download Perc,RP_D1_VMSEligible,RP_D1_ERPEligible,Society RP as per Sales Product,RP_D4,RP_HO,RP_D1
10695,1038800,2025-01-15,VMS only,VMS only,VMS only,,YTB,NaN,42.788462,Yes,,299705,1462240.5,1462240.5,1464926.4


#**Uniview Merge**

In [ ]:
dump_name = 'MIS-Tab1-Prospect-A new 16 dec based on last updated date'
Lead = pd.read_csv(file_path + DumpDate + "/Akash/"+ dump_name +".csv",parse_dates=True)
Lead = Lead[['lead_id', 'employee_code','created_on', 'close_date','source',]]

#Lead = Lead[~(Lead['employee_code'].isna())]
Lead[['created_on', 'close_date']] = Lead[['created_on', 'close_date']].apply(changeDateFormat)
Lead[['employee_code']] = Lead[['employee_code']].apply(cleanText)

Lead.rename({'lead_id':'Lead ID', 'employee_code':'Lead Created By','created_on':'Lead Creation Date', 'close_date':'Lead Closure Date',
       'source':'Lead Source'}, inplace= True, axis = 1)

###############################################

dump_name = 'MIS-Tab3-Closure'
ClosuresUniview = pd.read_csv(file_path + DumpDate + "/Akash/"+ dump_name +".csv",parse_dates=True)
ClosuresUniview = ClosuresUniview[['lead_id','closure_number','closure_date']]
ClosuresUniview[['closure_date']] = ClosuresUniview[['closure_date']].apply(changeDateFormat)
ClosuresUniview.rename({'lead_id':'Lead ID','closure_number':'Closure Number','closure_date':'Closure Date (Uniview)'}, inplace = True, axis = 1)

Lead = pd.merge(Lead, HR[['Emp Code','Emp Email ID','Role']], left_on = 'Lead Created By', right_on = 'Emp Code', how = 'left')
Lead = pd.merge(Lead, ClosuresUniview, on = 'Lead ID', how = 'right')

###############################################

dump_name = 'MIS-Tab2-Meeting-A with meeting start date filter'
meeting = pd.read_csv(file_path + DumpDate + "/Akash/"+ dump_name +".csv",parse_dates=True)
#meeting = pd.read_csv("/content/drive/MyDrive/Insights_Datalake/MIS-Tab2-Meeting-A with meeting start date filter -AVI.csv",parse_dates=True)

meeting = meeting[meeting['status'] == 'completed']
meeting = meeting[meeting['meeting_type'].isin({'mc_meeting','followup','sm_meeting','builder_meeting'})]

meeting = meeting[['Lead_id','meeting_id','start_date','end_date']]
meeting.rename({'Lead_id':'Lead ID','meeting_id':'Meeting ID', 'start_date':'Meeting Start Date','end_date':'Meeting End Date'}, inplace =True, axis = 1)
meeting[['Meeting Start Date','Meeting End Date']] = meeting[['Meeting Start Date','Meeting End Date']].apply(changeDateTimeFormat)

meeting['Meeting Start Time'] = meeting['Meeting Start Date'].dt.time
meeting['Total Meeting Duration (mins)'] = meeting['Meeting End Date'] - meeting['Meeting Start Date']
meeting['Total Meeting Duration (mins)'] = (meeting['Total Meeting Duration (mins)']/pd.Timedelta(minutes=1)).astype(int)

###
meeting = meeting[~(meeting['Total Meeting Duration (mins)'].isna())]
meeting = meeting[(meeting['Total Meeting Duration (mins)'] >= 10) & (meeting['Total Meeting Duration (mins)'] <= 240)]
###

meeting[['Meeting Start Date','Meeting End Date']] = meeting[['Meeting Start Date','Meeting End Date']].apply(changeDateFormat)
#meeting = meeting[meeting['Meeting Start Date'] == meeting['Meeting End Date']]
meeting.drop(['Meeting End Date'], inplace = True, axis = 1)
###

meetingSummary = meeting.groupby('Lead ID').agg({'Meeting ID':'count','Total Meeting Duration (mins)':'sum'}).reset_index()
meetingSummary.rename({'Meeting ID':'Completed Meeting Count'}, inplace = True, axis = 1)

###############################################
dump_name = 'Accounts Leads Ticket Tag'
ticketTag = pd.read_csv(file_path + DumpDate + "/Akash/"+ dump_name +".csv",parse_dates=True)
ticketTag = ticketTag[['base_lead__id','ticket_tag']]
ticketTag = ticketTag[ticketTag['ticket_tag'].notnull()]
ticketTag.rename(columns={'ticket_tag':'Ticket_Tag','base_lead__id':'Lead ID'},inplace = True)
ticketTag['Ticket_Tag'] = ticketTag['Ticket_Tag'].astype(str)
ticketTag_grp = ticketTag.groupby('Lead ID')['Ticket_Tag'].agg(lambda x: ', '.join(x)).reset_index()

###############################################

# dump_name = 'Questionnaire responses'
# DemoForm = pd.read_csv(file_path + DumpDate + "/Akash/"+ dump_name +".csv",parse_dates=True)
# #DemoForm = pd.read_csv("/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/Dec 2022/20 December 2022/Akash/Questionnaire responses.csv",parse_dates=True)

# DemoForm.rename({'Form added date':'Form Filled Date',  'form_added_by':'Form Filled By',
#                  'meeting__lead_id':'Lead ID', 'meeting__id':'Meeting ID', 'meeting__created_on':'Meeting Creation Date','meeting__end_date':'Meeting End Date', 'meeting__start_date':'Meeting Start Date',
#                  'meeting__meeting_title':'Meeting Title','meeting__meeting_type':'Meeting Client Type',
#                  'meeting__product_demo_given':'Product Demo Given', 'meeting__status':'Meeting Status'}, inplace = True, axis = 1)

# DemoForm[['Form Filled Date','Meeting Creation Date','Meeting Start Date','Meeting End Date']] = DemoForm[['Form Filled Date','Meeting Creation Date','Meeting Start Date','Meeting End Date']].apply(changeDateFormat)

# #DemoForm = DemoForm[(DemoForm['Form Filled Date'] == DemoForm['Meeting Start Date']) & (DemoForm['Meeting End Date'] == DemoForm['Meeting Start Date'])]
# DemoForm = DemoForm[DemoForm['Form template name'] == 'Demo form']

# DemoForm = DemoForm[['Lead ID','Meeting ID', 'Meeting Status', 'Meeting Client Type',
#                      'Meeting Creation Date','Meeting Start Date', 'Meeting End Date', 'Product Demo Given','mandatory',
#                      'Form Filled Date', 'Form Filled By',
#                      'question_label','question_data_type', 'text_value_response','numeric_value_response', 'date_value_response', 'bool_value_response']]

# DemoForm.loc[DemoForm['question_label'] == 'Enter POC phone no.', 'numeric_value_response'] = -1
# DemoForm['date_value_response'] = pd.to_numeric(DemoForm[(~DemoForm['date_value_response'].isna())]['date_value_response']).apply(lambda x: datetime.fromtimestamp(x/1000))
# DemoForm[['date_value_response']] = DemoForm[['date_value_response']].apply(changeDateFormat)

# ###############################################

# # dump_name = 'MIS-Tab6-Demo-Sheet with date filter'
# # DemoRec = pd.read_csv(file_path + DumpDate + "/Deepa Raghav/"+ dump_name +".csv",parse_dates=True)
# # DemoRec = DemoRec[['lead_id', 'meeting_id','call_recording']]
# # DemoRec.rename({'lead_id':'Lead ID','meeting_id':'Meeting ID','call_recording':'Recording Link'}, inplace = True, axis = 1)

# dump_name = 'Demo Recording (>15mins)'
# DemoRec = pd.read_csv(file_path + DumpDate + "/Akash/"+ dump_name +".csv",parse_dates=True)
# DemoRec = DemoRec[['lead_id', 'meeting_id','download_url']]
# DemoRec.rename({'lead_id':'Lead ID','meeting_id':'Meeting ID','download_url':'Recording Link'}, inplace = True, axis = 1)

# # DemoRec = DemoRec[['meeting_id','download_url']]
# # DemoRec.rename({'meeting_id':'Meeting ID','download_url':'Recording Link'}, inplace = True, axis = 1)

# ####################################################

# masterDemoRaw = pd.merge(DemoForm,DemoRec, left_on = ['Lead ID','Meeting ID'], right_on = ['Lead ID','Meeting ID'], how = 'left')
# # masterDemoRaw = pd.merge(DemoForm,DemoRec, left_on = ['Meeting ID'], right_on = ['Meeting ID'], how = 'left')

# masterDemoRaw = pd.merge(masterDemoRaw,HR, left_on = ['Form Filled By'], right_on = ['Emp Email ID'], how = 'left')

# masterDemoRaw = masterDemoRaw[['Form Filled Date', 'Form Filled By', 'Lead ID', 'Meeting ID',
#                                'Meeting Status', 'Meeting Client Type','Meeting Creation Date', 'Meeting Start Date', 'Meeting End Date', 'Product Demo Given', 'mandatory',
#                          'question_label', 'question_data_type', 'text_value_response','numeric_value_response', 'date_value_response', 'bool_value_response','Recording Link']]

# masterDemo = masterDemoRaw.copy()


# masterDemo = masterDemo[masterDemo['question_label'].isin({ 'Demo Participants',
#               'Enter demo duration ( in minutes)', 'Select mode of demo',
#        'Type of Demo', 'Select POC Designation',
#        'No. of people present during demo <society end>',
#        'Select demo date',
#        'Email ID of other TSM/ ASM', 'Span of Demo',
#        "Competitor active for (Note: Please ensure correct marking for multipliers. Any closures without declaration of competitor during demo won't be given multiplier)",
#        'Competitor active in the society at time of demo'})]
# masterDemo.loc[masterDemo['question_data_type'].str.contains('TEXT'), 'question_data_type'] = 'TEXT'
# masterDemo.loc[masterDemo['question_data_type'].str.contains('DATE'), 'question_data_type'] = 'DATE'
# masterDemo.loc[masterDemo['question_data_type'].str.contains('NUMBER'), 'question_data_type'] = 'NUMERIC'

# def fetchData(row):
#   if row['question_data_type'] == 'TEXT':
#     return row['text_value_response']
#   elif row['question_data_type'] == 'DATE':
#     return row['date_value_response']
#   elif row['question_data_type'] == 'NUMERIC':
#     return row['numeric_value_response']


# masterDemo['Demo Date']=masterDemo[masterDemo['question_label']=='Select demo date'].apply(fetchData,axis=1)
# masterDemo[['Demo Date']] = masterDemo[['Demo Date']].apply(changeDateFormat)
# masterDemo['Demo Date'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Date'].apply(lambda x: x.ffill().bfill())

# masterDemo['Demo Type']=masterDemo[masterDemo['question_label']=='Type of Demo'].apply(fetchData,axis=1)
# masterDemo['Demo Type'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Type'].apply(lambda x: x.ffill().bfill())

# masterDemo['Demo Society POC']=masterDemo[masterDemo['question_label']=='Select POC Designation'].apply(fetchData,axis=1)
# masterDemo['Demo Society POC'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Society POC'].apply(lambda x: x.ffill().bfill())

# masterDemo['Demo Participants']=masterDemo[masterDemo['question_label']=='Demo Participants'].apply(fetchData,axis=1)
# masterDemo.loc[masterDemo['Demo Participants'].isin({',SM','SM'}), 'Demo Participants'] = 'SM'
# masterDemo.loc[masterDemo['Demo Participants'].isin({',MC','MC'}), 'Demo Participants'] = 'MC'
# masterDemo.loc[masterDemo['Demo Participants'].isin({'SM,MC', 'MC,SM', ',SM,MC',',MC,SM'}), 'Demo Participants'] = 'MC + SM'
# masterDemo['Demo Participants'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Participants'].apply(lambda x: x.ffill().bfill())


# masterDemo['Demo Duration (in mins)']=masterDemo[masterDemo['question_label']=='Enter demo duration ( in minutes)'].apply(fetchData,axis=1)
# masterDemo['Demo Duration (in mins)'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Duration (in mins)'].apply(lambda x: x.ffill().bfill())

# masterDemo['Demo Mode']=masterDemo[masterDemo['question_label']=='Select mode of demo'].apply(fetchData,axis=1)
# masterDemo['Demo Mode'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Mode'].apply(lambda x: x.ffill().bfill())

# masterDemo['Demo Participant Count']=masterDemo[masterDemo['question_label']=='No. of people present during demo <society end>'].apply(fetchData,axis=1)
# masterDemo['Demo Participant Count'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Participant Count'].apply(lambda x: x.ffill().bfill())

# masterDemo['Demo Team Type']=masterDemo[masterDemo['question_label']=='Span of Demo'].apply(fetchData,axis=1)
# masterDemo['Demo Team Type'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Team Type'].apply(lambda x: x.ffill().bfill())

# # masterDemo['Demo Team Members']=masterDemo[masterDemo['question_label']=='Email ID of other TSM/ ASM'].apply(fetchData,axis=1)
# # masterDemo['Demo Team Members'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Team Members'].apply(lambda x: x.ffill().bfill())

# masterDemo['Competitor Name']=masterDemo[masterDemo['question_label']=='Competitor active in the society at time of demo'].apply(fetchData,axis=1)
# masterDemo['Competitor Name'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Competitor Name'].apply(lambda x: x.ffill().bfill())

# masterDemo['Competitor Product']=masterDemo[masterDemo['question_label']=='Competitor active for (Note: Please ensure correct marking for multipliers. Any closures without declaration of competitor during demo won\'t be given multiplier)'].apply(fetchData,axis=1)
# masterDemo['Competitor Product'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Competitor Product'].apply(lambda x: x.ffill().bfill())


# masterDemo.drop(['mandatory','question_label','question_data_type', 'text_value_response', 'numeric_value_response',
#        'date_value_response', 'bool_value_response'], axis = 1, inplace=True)

# masterDemo = masterDemo.drop_duplicates()

wb = gc.open_by_key("1TGX2XuuzC_kZD_H30fdqWxD4GXiVdYY6HUdXU2lOS4c")
ws = wb.worksheet("masterDEMO")
md = ws.get_all_values()
Demo = pd.DataFrame(md)
Demo.columns = Demo.iloc[0]
Demo = Demo.drop(0).reset_index()

Demo.rename({'Meeting Start Date':'Demo Date','Recording Link_Uniview':'Recording Link'}, inplace = True, axis = 1)
Demo['Demo Duration (in mins)'] = ""
Demo['Demo Duration (in mins)'] = pd.to_numeric(Demo['Demo Duration (in mins)'])
masterDemo = Demo[['Form Filled Date', 'Form Filled By', 'Meeting ID', 'Demo Date', 'Demo Type','Demo Mode', 'Demo Duration (in mins)', 'Recording Link']]


<ipython-input-167-687cdeefc823>:2: DtypeWarning: Columns (0,24) have mixed types. Specify dtype option on import or set low_memory=False.
  Lead = pd.read_csv(file_path + DumpDate + "/Akash/"+ dump_name +".csv",parse_dates=True)


In [ ]:
meeting1 = pd.merge(meeting, masterDemo, on = 'Meeting ID', how = 'left')

meeting1 = meeting1[~meeting1['Recording Link'].isna()]
meeting1 = meeting1.sort_values(['Demo Duration (in mins)']).groupby(['Lead ID']).tail(1)

meeting2 = pd.merge(meeting, masterDemo, on = 'Meeting ID', how = 'left')

meetingSummary = meeting2.groupby('Lead ID').agg({'Meeting ID':'count','Total Meeting Duration (mins)':'mean','Form Filled Date':'count','Demo Duration (in mins)':'mean'}).reset_index()
meetingSummary.rename({'Meeting ID':'Completed Meeting Count','Form Filled Date':'Demos Given'}, inplace = True, axis = 1)

meetingSummary = pd.merge(meetingSummary,meeting1[['Lead ID','Form Filled By','Demo Date','Recording Link']], on = 'Lead ID', how = 'left')

leadMaster = pd.merge(Lead,meetingSummary, on = 'Lead ID', how ='left')
leadMaster = pd.merge(leadMaster,ticketTag_grp, on = 'Lead ID', how ='left')

leadMaster = leadMaster[leadMaster['Lead Closure Date'] >= pd.to_datetime("2021-08-01").date()]

In [ ]:
closures = pd.merge(leadMaster,closures, on = 'Closure Number', how = 'right')

In [ ]:
closures

,Lead ID,Lead Created By,Lead Creation Date,Lead Closure Date,Lead Source,Emp Code,Emp Email ID,Role,Closure Number,Closure Date (Uniview),...,Monetization Multiplier,Estimated Device Cost,RP_D1_ERPEligible,RP_D1_VMSEligible,D1 Product,RP_D1,PrimaryRPsHO,SecondaryRPsHO,PrimaryRPsD1,SecondaryRPsD1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1034080,NaN,...,1.0,9900,Yes,Yes,Both VMS + ERP,2875.0,5198.0,0.0,2875.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1034084,NaN,...,1.0,9900,,Yes,VMS only,37436.0,35612.0,0.0,37436.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1034091,NaN,...,1.0,9900,Yes,,ERP only,14925.0,9619.0,0.0,14925.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1034104,NaN,...,1.0,9900,No,No,NaN,NaN,0.0,0.0,NaN,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1034100,NaN,...,1.0,19800,,Yes,VMS only,53087.0,16863.5,16863.5,26543.5,26543.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4457,542cf6baabb74970a0ffc4df82419983,NBTS18134,2024-12-11,2025-02-28,City,NBTS18134,swapnal.sunil@nobroker.in,TSM,1039662,2025-02-28,...,1.0,9900,No,No,NaN,NaN,0.0,0.0,NaN,0.0
4458,2ca498dc205d45cab03188083de4ac4b,NBTS32627,2025-02-11,2025-03-01,referral_from_admin_app_signup,NBTS32627,anshu.kumari@nobroker.in,CENTRAL SALES ASSOCIATE,1039663,2025-03-01,...,1.0,19800,No,No,NaN,0.0,0.0,0.0,0.0,0.0
4459,211870610d45465db152904c31f1eb37,NBTS14269,2025-02-05,2025-02-21,City,NBTS14269,prasad.chavan@nobroker.in,AASM,1039520,2025-02-21,...,1.0,9900,No,No,NaN,NaN,0.0,0.0,NaN,NaN
4460,74dc4f1c0dc64b828c5f59c01cff7043,NBTS30424,2025-02-16,2025-02-28,nbh_website,NBTS30424,debabrata.pandey@nobroker.in,CENTRAL SALES ASSOCIATE,1039660,2025-02-28,...,1.0,9900,No,No,NaN,NaN,0.0,0.0,NaN,0.0


#**📜Sales Revenue**

###**Old Data Revenue**

In [ ]:
spreadSheetId = "1yfBhbPRbbJ1pqAXd3nTgOjwFLlLL2cAfvquAq9-70NM"
rangeName = "Live_Data"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
rev = get_data_df(spreadSheetId, rangeName)

rev.columns=rev.iloc[3,:]
rev = rev.iloc[4:,:]
rev_lead = rev[['Timestamp','City','Primary TSM Email id','Secondary TSM email id','ASM email address','Product Sold','Closure Number','Society Name (Sales)','The amount collected including GST','Confirmation form Finance',
                'Date of Transaction','REC Code','Ref ID']]

#rev_lead.loc[:,'Closure Number'] = pd.to_numeric(rev_lead['Closure Number'], errors = 'coerce')
#rev_lead.loc[:,'The amount collected including GST'] = pd.to_numeric(rev_lead['The amount collected including GST'])
rev_lead.loc[:,'Date of Transaction'] = pd.to_datetime(rev_lead['Date of Transaction'], errors = 'coerce').dt.date
rev_lead['Timestamp'] = pd.to_datetime(rev_lead['Timestamp'],format = '%m/%d/%Y %H:%M:%S')
# rev_lead['Timestamp'] = rev_lead['Timestamp'].dt.strftime('%m/%d/%Y %H:%M:%S')

rev_lead['The amount collected including GST_Finance'] = rev_lead['The amount collected including GST']
rev_lead['The amount collected including GST_Finance'] = rev_lead['The amount collected including GST_Finance'].str.replace(",", "")
rev_lead['The amount collected including GST_Finance'] = pd.to_numeric(rev_lead['The amount collected including GST_Finance'])

old_rev = rev_lead[(rev_lead['Date of Transaction']<pd.to_datetime("2023-07-01").date())]

<ipython-input-171-ab241092d44a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rev_lead['Timestamp'] = pd.to_datetime(rev_lead['Timestamp'],format = '%m/%d/%Y %H:%M:%S')
<ipython-input-171-ab241092d44a>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rev_lead['The amount collected including GST_Finance'] = rev_lead['The amount collected including GST']
<ipython-input-171-ab241092d44a>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [ ]:
### Till Sep'24 Static Revenue Form Response ###

spreadSheetId = "1UFXWeAWt017XAtEzl0EKxd-qjC4T6IeJYdzWWxLLOxo"
rangeName = "Form Responses"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
rev_sep = get_data_df(spreadSheetId, rangeName)
rev_sep.columns=rev_sep.iloc[0,:]
rev_sep = rev_sep.iloc[1:,:]
rev_sep = rev_sep.drop([1])
rev_sep = rev_sep[(rev_sep['Select Team']=='Sales') | (rev_sep['Select Team']=='sales') | (rev_sep['Select Team']=='SALES')]
def convert_to_date(row):
    try:
        return datetime.strptime(row, "%b'%y")
    except ValueError:
        return ""

# Apply the conversion function to the DataFrame
rev_sep['Revenue Month_Finance'] = rev_sep['Revenue Considered Month'].apply(convert_to_date)
rev_sep['Timestamp'] = pd.to_datetime(rev_sep['Timestamp'],format = '%d/%m/%Y %H:%M:%S')
# rev_sep['Timestamp'] = rev_sep['Timestamp'].dt.strftime('%d/%m/%Y %H:%M:%S')

rev_sep.loc[rev_sep['Product Sold'] == 'Boom barrier','Product Sold'] = 'Boom Barriers/RFIDs'
rev_sep.rename({'Primary Employee Email ID':'Primary TSM Email id','Secondary Employee Email ID':'Secondary TSM email id', 'ASM/BAM email address':'ASM email address', 'Society  Name':'Society Name (Sales)',
                'City (Society)':'City','Confirmation':'Confirmation form Finance','Transaction Date':'Date of Transaction','Received Amount':'The amount collected including GST','"Received Amount"':'The amount collected including GST_Finance', 'Hood Rev Code':'Ref ID', 'Discrepency Remarks':'Remarks'},inplace=True, axis=1)

rev_sep.loc[rev_sep['Closure Number'].isin(['1033886','1030301','1034078','1034132']),'City']='Mumbai'
rev_sep.loc[rev_sep['Ref ID'].isin(['Hood-Rev-2669','Hood-Rev-3308']),'The amount collected including GST_Finance']= '19,116'

rev_sep.loc[rev_sep['Ref ID'].isin(['Hood-Rev-3308','Hood-Rev-4523','Hood-Rev-4569','Hood-Rev-4638','Hood-Rev-4318','Hood-Rev-5137','Hood-Rev-5183','Hood-Rev-4619','Hood-Rev-5044','Hood-Rev-5345','Hood-Rev-5605','Hood-Rev-5682','Hood-Rev-5672','Hood-Rev-5608','Hood-Rev-6005','Hood-Rev-6145','Hood-Rev-6756','Hood-Rev-8044','Hood-Rev-8838']),'The amount collected including GST_Finance']= rev_sep['The amount collected including GST']
rev_sep.loc[rev_sep['Ref ID'].isin(['Hood-Rev-2669','Hood-Rev-3308','Hood-Rev-4523','Hood-Rev-4569','Hood-Rev-4638','Hood-Rev-4318','Hood-Rev-5137','Hood-Rev-5183','Hood-Rev-4619','Hood-Rev-5044','Hood-Rev-5345','Hood-Rev-5605','Hood-Rev-5682','Hood-Rev-5672','Hood-Rev-5608','Hood-Rev-6005','Hood-Rev-6145','Hood-Rev-6756','Hood-Rev-8044','Hood-Rev-8838']),'Confirmation form Finance']= "Received"
rev_sep.loc[rev_sep['Ref ID'].isin(['Hood-Rev-2669','Hood-Rev-3308','Hood-Rev-4523','Hood-Rev-4569','Hood-Rev-4638','Hood-Rev-4318','Hood-Rev-5137','Hood-Rev-5183','Hood-Rev-4619','Hood-Rev-5044','Hood-Rev-5345','Hood-Rev-5605','Hood-Rev-5682','Hood-Rev-5672','Hood-Rev-5608','Hood-Rev-6005','Hood-Rev-6145','Hood-Rev-6756','Hood-Rev-8044','Hood-Rev-8838']),'Date of Transaction']= rev_sep['Paid Date (Deposit Date in case of cheque)']
rev_sep.loc[rev_sep['Ref ID'].isin(['Hood-Rev-2669','Hood-Rev-3308','Hood-Rev-4523','Hood-Rev-4569','Hood-Rev-4638','Hood-Rev-4318','Hood-Rev-5137','Hood-Rev-5183','Hood-Rev-4619','Hood-Rev-5044','Hood-Rev-5345','Hood-Rev-5605','Hood-Rev-5682','Hood-Rev-5672','Hood-Rev-5608','Hood-Rev-6005','Hood-Rev-6145','Hood-Rev-6756','Hood-Rev-8044','Hood-Rev-8838']),'Revenue Month_Finance']= rev_sep['Paid Date (Deposit Date in case of cheque)']

static_rev = rev_sep[['Timestamp','City','Primary TSM Email id','Secondary TSM email id','ASM email address','Product Sold','Closure Number','Society Name (Sales)','The amount collected including GST','Confirmation form Finance','Date of Transaction','REC Code',
                   'Ref ID','The amount collected including GST_Finance','Remarks','Revenue Month_Finance','Paid Date (Deposit Date in case of cheque)']]

static_rev['The amount collected including GST_Finance'] = static_rev['The amount collected including GST_Finance'].str.replace(",", "")
static_rev['The amount collected including GST_Finance'] = pd.to_numeric(static_rev['The amount collected including GST_Finance'])

<ipython-input-172-bf6cec73340f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rev_sep['Revenue Month_Finance'] = rev_sep['Revenue Considered Month'].apply(convert_to_date)
<ipython-input-172-bf6cec73340f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rev_sep['Timestamp'] = pd.to_datetime(rev_sep['Timestamp'],format = '%d/%m/%Y %H:%M:%S')
<ipython-input-172-bf6cec73340f>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentati

In [ ]:
# Brand Provided data
data_dict = {
    'Timestamp': ['3/06/2024 0:00:00','6/05/2024 0:00:00','6/05/2024 0:00:00','6/05/2024 0:00:00'],
    'City': ['Pune','Pune','Pune','Pune'],
    'Primary TSM Email id': ['ahya.kishan@nobroker.in','dhiraj.bochare@nobroker.in','daud.khan@nobroker.in','sufiyan.salim@nobroker.in'],
    'Secondary TSM email id': ['',"","",""],
    'ASM email address': ['sushant.jawale@nobroker.in','vikas.suryavanshi@nobroker.in','vikas.suryavanshi@nobroker.in','vikas.suryavanshi@nobroker.in'],
    'Product Sold': ['Brand Referral','Brand Referral','Brand Referral','Brand Referral'],
    'Closure Number': [1020000,1020000,1020000,1020000],
    'Society Name (Sales)': ['Orchid International School','krishna buildcon & choice lifestyle','Biophilias plant','Orbis school & Dr Lal Path Lab'],
    'The amount collected including GST': [59000,17741,2336,17081],
    'Confirmation form Finance': ['Received','Received','Received','Received'],
    'Date of Transaction': [datetime(2024, 2, 29),datetime(2024, 5, 31),datetime(2024, 5, 31),datetime(2024, 5, 31)],
    'REC Code': ['-','-','-','-'],
    'Ref ID': ['Brand-1','Brand-2','Brand-3','Brand-4'],
    'The amount collected including GST_Finance': [59000,17741,2336,17081],
    'Remarks': ['Brand Referral - Orchid International School','Brand Referral','Brand Referral','Brand Referral'],
    'Revenue Month_Finance': [datetime(2024, 2, 1),datetime(2024, 5, 1),datetime(2024, 5, 1),datetime(2024, 5, 1)]
}

# Create a DataFrame
Brand_Referral = pd.DataFrame(data_dict)

Brand_Referral['Timestamp'] = pd.to_datetime(Brand_Referral['Timestamp'],format = '%m/%d/%Y %H:%M:%S')

# Display the DataFrame
Brand_Referral.shape

(4, 16)

###**New Revenue Data**

In [ ]:
### New Revenue Form Response ###

spreadSheetId = "12Rt_yAH_z3dFVasLWUVsdTfxVewFQsJ3FgNCxYKcA-4"
rangeName = "Form Response"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
new_rev = get_data_df(spreadSheetId, rangeName)

new_rev.columns=new_rev.iloc[0,:]
new_rev = new_rev.iloc[1:,:]
new_rev = new_rev.drop([1])
new_rev.shape

(992, 73)

In [ ]:
# Apply the month conversion function to the New_Rev DataFrame
new_rev['Revenue Month_Finance'] = new_rev['Revenue Month'].apply(convert_to_date)
new_rev['Timestamp'] = pd.to_datetime(new_rev['Timestamp'],format = '%m/%d/%Y %H:%M:%S')
# new_rev['Timestamp'] = new_rev['Timestamp'].dt.strftime('%m/%d/%Y %H:%M:%S')

new_rev = new_rev[['Timestamp','City','Primary TSM Email id','Secondary TSM email id','ASM email address','Product Sold','Closure Number','Society Name (Sales)','The amount collected including GST','Confirmation form Finance','Date of Transaction','REC Code',
                   'Ref ID','The amount collected including GST_Finance','Remarks','Revenue Month_Finance','Paid Date (Deposit Date in case of cheque)']]

new_rev.loc[new_rev['Product Sold'] == 'Boom barrier','Product Sold'] = 'Boom Barriers/RFIDs'

# new_rev['The amount collected including GST_Finance'] = new_rev['The amount collected including GST_Finance'].str.replace(",", "")
# new_rev['The amount collected including GST_Finance'] = pd.to_numeric(new_rev['The amount collected including GST_Finance'])

new_rev.loc[new_rev['Ref ID'].isin(['Hood-Rev-9905','Hood-Rev-11624']),'The amount collected including GST_Finance']= new_rev['The amount collected including GST']
new_rev.loc[new_rev['Ref ID'].isin(['Hood-Rev-9905','Hood-Rev-11624']),'Confirmation form Finance']= "Received"
new_rev.loc[new_rev['Ref ID'].isin(['Hood-Rev-9905','Hood-Rev-11624']),'Date of Transaction']= new_rev['Paid Date (Deposit Date in case of cheque)']
new_rev.loc[new_rev['Ref ID'].isin(['Hood-Rev-9905','Hood-Rev-11624']),'Revenue Month_Finance']= new_rev['Paid Date (Deposit Date in case of cheque)']

new_rev['The amount collected including GST_Finance'] = new_rev['The amount collected including GST_Finance'].str.replace(",", "")
new_rev['The amount collected including GST_Finance'] = pd.to_numeric(new_rev['The amount collected including GST_Finance'])

rev = pd.concat([new_rev,static_rev,Brand_Referral,old_rev])

rev['Revenue Month_Finance'].fillna(rev['Date of Transaction'], inplace=True)

rev[['City']] = rev[['City']].apply(properText)
rev[['Primary TSM Email id','Secondary TSM email id','ASM email address']] = rev[['Primary TSM Email id','Secondary TSM email id','ASM email address']].apply(lowerText)

rev[['Revenue Month_Finance']] = rev[['Revenue Month_Finance']].apply(changeDateFormat)
rev.reset_index(drop=True, inplace=True)

<ipython-input-175-f2880c81f43f>:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  rev['Revenue Month_Finance'].fillna(rev['Date of Transaction'], inplace=True)


In [ ]:
rev.drop_duplicates('Ref ID',keep='last',inplace=True)

In [ ]:
columns_to_update = ['The amount collected including GST_Finance', 'Confirmation form Finance', 'Date of Transaction', 'Revenue Month_Finance', 'REC Code']
for col in columns_to_update:
    rev.loc[rev['Ref ID'].isin(['Hood-Rev-8858', 'Hood-Rev-8947','Hood-Rev-8935','Hood-Rev-9040','Hood-Rev-8977','Hood-Rev-9022','Hood-Rev-8878','Hood-Rev-9008','Hood-Rev-9037']), col] = new_rev.loc[new_rev['Ref ID'].isin(['Hood-Rev-8858',
                                'Hood-Rev-8947','Hood-Rev-8935','Hood-Rev-9040','Hood-Rev-8977','Hood-Rev-9022','Hood-Rev-8878','Hood-Rev-9008','Hood-Rev-9037']), col].values

In [ ]:
##### Contribution share code block

#rev['The amount collected including GST_Finance'] = rev['The amount collected including GST_Finance'].astype(str).apply(lambda x: x.replace(",", ""))
rev['The amount collected including GST_Finance'] = pd.to_numeric(rev['The amount collected including GST_Finance'])

rev.loc[rev['Ref ID'].isin(['Hood-Rev-4523','Hood-Rev-4569','Hood-Rev-4318','Hood-Rev-5183','Hood-Rev-4619','Hood-Rev-5044','Hood-Rev-5608','Hood-Rev-5411','Hood-Rev-6005','Hood-Rev-6756','Hood-Rev-8044']),'The amount collected including GST_Finance']= rev['The amount collected including GST_Finance']/2

rev.loc[:,'Closure Number'] = pd.to_numeric(rev['Closure Number'], errors = 'coerce')
rev['Amount collected excluding GST'] = round((rev['The amount collected including GST_Finance']/1.18),2)

In [ ]:
dump_name = 'Final Master Data'
kibanaMap = pd.read_csv(file_path + DumpDate + "/Master/"+ dump_name +".csv",parse_dates=True)
kibanaMap.rename({'kibanaID':'Kibana ID','closureID.Uniview':'Closure Number', 'societyStatus':'Society Status'}, inplace = True, axis = 1)
kibanaMap.loc[:,'Closure Number'] = pd.to_numeric(kibanaMap['Closure Number'], errors = 'coerce')
kibanaMap = kibanaMap[~kibanaMap['Closure Number'].isna()]
kibanaMap = kibanaMap[['Closure Number','Kibana ID','Society Status']]

kibanaMap = kibanaMap.drop_duplicates(subset=['Kibana ID'],keep ='first')
kibana_merge = pd.merge(rev,kibanaMap,how = 'left',on = "Closure Number")

<ipython-input-179-28baa964387d>:2: DtypeWarning: Columns (10,17,18,20,34,51) have mixed types. Specify dtype option on import or set low_memory=False.
  kibanaMap = pd.read_csv(file_path + DumpDate + "/Master/"+ dump_name +".csv",parse_dates=True)


In [ ]:
dump_name = 'LeadMaster'
sales_master = pd.read_csv(file_path + DumpDate + "/Master/"+ dump_name +".csv",parse_dates=True)
sales_master = sales_master[['Closure Number','Download Perc']]
sales_master = sales_master.drop_duplicates(subset=['Closure Number'],keep ='last')

rev_merge = pd.merge(kibana_merge,sales_master,how = 'left',on = "Closure Number")
rev_merge.loc[rev_merge['City'] == 'Thane','City'] = 'Navi Mumbai'

<ipython-input-180-d50d84b8f7ae>:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,14,15,16,51,52,53,54,55,63,106,118,119,122,130) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_master = pd.read_csv(file_path + DumpDate + "/Master/"+ dump_name +".csv",parse_dates=True)


In [ ]:
rev_merge = rev_merge[['Timestamp', 'City', 'Primary TSM Email id', 'Secondary TSM email id','ASM email address', 'Product Sold', 'Closure Number','Society Name (Sales)',
       'The amount collected including GST', 'Confirmation form Finance','Date of Transaction', 'REC Code', 'Ref ID','The amount collected including GST_Finance',
       'Amount collected excluding GST','Remarks','Revenue Month_Finance', 'Kibana ID', 'Society Status','Download Perc']]

rev_merge = rev_merge.drop_duplicates()

###**Write to Sheet**

In [ ]:
spreadsheetid = "1yfBhbPRbbJ1pqAXd3nTgOjwFLlLL2cAfvquAq9-70NM"
range = 'Revenue_Master'
wb = gc.open_by_key(spreadsheetid)
ws = wb.worksheet(range)
df = pd.DataFrame(rev_merge)

wb.values_clear("Revenue_Master!A:T")
set_with_dataframe(ws, df.sort_values(by=['Timestamp'], ascending = True))

#**Write to Sheet**#

In [ ]:
closures = closures[['Lead ID','Lead Created By','Lead Creation Date','Lead Closure Date','Lead Source','Ticket_Tag','Emp Code','Emp Email ID','Role',
                     'Completed Meeting Count','Total Meeting Duration (mins)', 'Demos Given', 'Demo Duration (in mins)', 'Form Filled By', 'Demo Date','Recording Link',
                     "Closure Type", "Closure Date", "Original Closure Number", "Closure Number", "Closure Form Filled Date", "Closure Form Filled Time", "Closure Month",
                     "Closure Form Filled By", "Team Type", "Lead Type", "Closure Done By (Primary Email)", "Primary BD City", "Primary BD Role", "Closure Done By (Secondary Email)",
                     "Secondary BD City", "Secondary BD Role", "Society Type", "Property Type", "VMSTrialPeriod", "ERPTrialPeriod", "Society Name (Sales)", "Closure City", "City Type",
                     "Total Flats", "VMS Sold Flats", "ERP Sold Flats", "Sold Flats", "Occupancy Percentage", "Pin Code", "Society Latitude", "Society Longitude",
                     "Rent of the society", "New Devices", "Refurbished Devices", "Subscription Revenue", "Upfront Revenue (One time)", "Access control/Boom Barrier net revenue",
                     "Welfare Amount", "Monetization Advance Amount", "RP (Revenue Points) to be claimed", "Revenue to be claimed", "WrongPrimaryBDMail", "WrongSecondaryBDMail", "WrongHOProduct",
                     "CrossCityClosure", "DuplicateClosureNumber", "WrongOriginalClosureNumber", "ClosureActivity", "VMSHOFilledDate", "VMSHOFilledEmail", "VMSHOMeetingID", "VMSHODate",
                     "Occupied Flats (VMS)", "Sold Flats (VMS)", "VMSHOStatus", "VMS Handover Type", "Obstruction in VMS HO", "VMS SKU ID", "VMS HO Done by", "ERPHOFilledDate", "ERPHOFilledEmail",
                     "ERPHOMeetingID", "ERPHODate", "Occupied Flats (ERP)", "Sold Flats (ERP)", "ERPHOStatus", "ERP Handover Type", "Obstruction in ERP HO", "ERP SKU ID", "ERP HO Done by",
                     "VMS Sold Flats Mismatch", "VMSHOClosureFilledTAT", "VMSHOClosureTAT", "ERP Sold Flats Mismatch", "ERPHOClosureFilledTAT", "ERPHOClosureTAT", "Closure Form Filling TAT",
                     "New device cost", "Refurbished device cost", "Monetization cost", "Sales cost", "Intercept Value", "Flat Count as per occupancy", "Flat Count Value",
                     "Locality", "Locality value", "Premiumness", "Percentage of median rent", "Premiumness Value", "Revenue (Upfront, Subscription, Access Control/ Boom Barrier)",
                     "Agreement Validity", "Welfare Amount - PIA", "Closure Type - PIA", "Society Latitude - PIA", "Society Longitude - PIA", "Rent of the Society - PIA", "New Devices - PIA",
                     "Refurbished Devices - PIA", "VMS Sold Flats - PIA", "ERP Sold Flats - PIA", "Occupancy % - PIA", "Occupancy - PIA","Flat Count as per occupancy - PIA", "Flat Count Value - PIA",
                     "Locality - PIA", "Locality Value - PIA", "Premiumness - PIA", "Percentage of median rent - PIA", "Premiumness Value - PIA","Society RP Eligibility","Sales Product", "HO Product","D1 Product",
                     "Society RP as per Sales Product", "Society RP as per HO Product","RP_D4", "RP_HO",'RP_D1','PrimaryRPsHO','SecondaryRPsHO','PrimaryRPsD1','SecondaryRPsD1','Download Count','Download Perc',
                     'combined IPR','RP_D1_VMSEligible','RP_D1_ERPEligible','Kibana ID','Kibana Name','Auto-Matched Kibana ID','Kibana Creation Date','Onb Status','Onb Date','Bank Addition Date','Society AccountStatus',
                     'Posted Date',	'Deletion Status','Bill Types',	'Bill Frequency',	'Flat Count',	'Total Posted Amount in K',	'First Posting',	'Latest Posting',	'Payment Unique Flats',	'Total Payment Amount in K',
                     "monthDays", "year", "HO Meeting ID", "Current VMS software used", "Current ERP software used", "Closure Proof Link", "Agreement Type", "Closure Status","Team Category",'Gross RP Without Offset',
                     'VMS Sold Flats as per Occupancy','Competition Multiplier (PIA) - VMS','Competition Multiplier (PIA) - ERP','VMSTrialReductionFactor','Date_Multiplier','Brand New Devices','Biometrics Devices',
                     'Gross RP - PIA_Validated RP Component', 'Gross RP - PIA_Only Valid', 'Gross RP - PIA as per D1 Product','Gross RP - PIA_Validated RP Component Onboarded',
                     'VMS Onb Status','ERP Onb Status','Onb Product_Current Signing','Agreement_Multiplier']]

closures.drop_duplicates(inplace=True)

closures = closures.sort_values(by=['Closure Date'], ascending = False)
closures["Last Updated Datetime"] = pd.to_datetime(datetime.now()).tz_localize('UTC').tz_convert('Asia/Kolkata').strftime("%d-%b-%Y - %H:%M")
closures = closures.replace(np.nan,None)
closures = closures.replace(np.inf,None)

closures[['Closure Date']] = closures[['Closure Date']].apply(changeDateFormat)

closures.shape

<ipython-input-183-36c89d713c7a>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closures.drop_duplicates(inplace=True)


(4462, 187)

In [ ]:
closures = closures[['Lead ID','Lead Created By','Lead Creation Date','Lead Closure Date','Lead Source','Ticket_Tag','Emp Code','Emp Email ID','Role',
                     'Completed Meeting Count','Total Meeting Duration (mins)', 'Demos Given', 'Demo Duration (in mins)', 'Form Filled By', 'Demo Date','Recording Link',
                     "Closure Type", "Closure Date", "Original Closure Number", "Closure Number", "Closure Form Filled Date", "Closure Form Filled Time", "Closure Month",
                     "Closure Form Filled By", "Team Type", "Lead Type", "Closure Done By (Primary Email)", "Primary BD City", "Primary BD Role", "Closure Done By (Secondary Email)",
                     "Secondary BD City", "Secondary BD Role", "Society Type", "Property Type", "VMSTrialPeriod", "ERPTrialPeriod", "Society Name (Sales)", "Closure City", "City Type",
                     "Total Flats", "VMS Sold Flats", "ERP Sold Flats", "Sold Flats", "Occupancy Percentage", "Pin Code", "Society Latitude", "Society Longitude",
                     "Rent of the society", "New Devices", "Refurbished Devices", "Subscription Revenue", "Upfront Revenue (One time)", "Access control/Boom Barrier net revenue",
                     "Welfare Amount", "Monetization Advance Amount", "RP (Revenue Points) to be claimed", "Revenue to be claimed", "WrongPrimaryBDMail", "WrongSecondaryBDMail", "WrongHOProduct",
                     "CrossCityClosure", "DuplicateClosureNumber", "WrongOriginalClosureNumber", "ClosureActivity", "VMSHOFilledDate", "VMSHOFilledEmail", "VMSHOMeetingID", "VMSHODate",
                     "Occupied Flats (VMS)", "Sold Flats (VMS)", "VMSHOStatus", "VMS Handover Type", "Obstruction in VMS HO", "VMS SKU ID", "VMS HO Done by", "ERPHOFilledDate", "ERPHOFilledEmail",
                     "ERPHOMeetingID", "ERPHODate", "Occupied Flats (ERP)", "Sold Flats (ERP)", "ERPHOStatus", "ERP Handover Type", "Obstruction in ERP HO", "ERP SKU ID", "ERP HO Done by",
                     "VMS Sold Flats Mismatch", "VMSHOClosureFilledTAT", "VMSHOClosureTAT", "ERP Sold Flats Mismatch", "ERPHOClosureFilledTAT", "ERPHOClosureTAT", "Closure Form Filling TAT",
                     "New device cost", "Refurbished device cost", "Monetization cost", "Sales cost", "Intercept Value", "Flat Count as per occupancy", "Flat Count Value",
                     "Locality", "Locality value", "Premiumness", "Percentage of median rent", "Premiumness Value", "Revenue (Upfront, Subscription, Access Control/ Boom Barrier)",
                     "Agreement Validity", "Welfare Amount - PIA", "Closure Type - PIA", "Society Latitude - PIA", "Society Longitude - PIA", "Rent of the Society - PIA", "New Devices - PIA",
                     "Refurbished Devices - PIA", "VMS Sold Flats - PIA", "ERP Sold Flats - PIA", "Occupancy % - PIA", "Occupancy - PIA","Flat Count as per occupancy - PIA", "Flat Count Value - PIA",
                     "Locality - PIA", "Locality Value - PIA", "Premiumness - PIA", "Percentage of median rent - PIA", "Premiumness Value - PIA","Society RP Eligibility","Sales Product", "HO Product","D1 Product",
                     "Society RP as per Sales Product", "Society RP as per HO Product","RP_D4", "RP_HO",'RP_D1','PrimaryRPsHO','SecondaryRPsHO','PrimaryRPsD1','SecondaryRPsD1','Download Count','Download Perc',
                     'combined IPR','RP_D1_VMSEligible','RP_D1_ERPEligible','Kibana ID','Kibana Name','Auto-Matched Kibana ID','Kibana Creation Date','Onb Status','Onb Date','Bank Addition Date','Society AccountStatus',
                     'Posted Date',	'Deletion Status','Bill Types',	'Bill Frequency',	'Flat Count',	'Total Posted Amount in K',	'First Posting',	'Latest Posting',	'Payment Unique Flats',	'Total Payment Amount in K',
                     "monthDays", "year", "HO Meeting ID", "Current VMS software used", "Current ERP software used", "Closure Proof Link", "Agreement Type", "Closure Status","Team Category",'Gross RP Without Offset',
                     'VMS Sold Flats as per Occupancy','Competition Multiplier (PIA) - VMS','Competition Multiplier (PIA) - ERP','VMSTrialReductionFactor','Date_Multiplier','Brand New Devices','Biometrics Devices',
                     'Last Updated Datetime','Gross RP - PIA_Validated RP Component', 'Gross RP - PIA_Only Valid', 'Gross RP - PIA as per D1 Product','Gross RP - PIA_Validated RP Component Onboarded',
                     'VMS Onb Status','ERP Onb Status','Onb Product_Current Signing','Agreement_Multiplier']]

In [ ]:
spreadSheetId = "1BBIaBIbkKuQfVYt8W8VwbRPiztg29HPuBHzW764rEwE"
#rangeName = "ClosureHOMaster"
rangeName = "LeadMaster"

wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
#ws.clear()
wb.values_clear("LeadMaster!A:GE")

# APPEND DATA TO SHEET
set_with_dataframe(ws, closures)

In [ ]:
spreadSheetId="1p9VJ0ATPxz68WMfR4uY0ZFb7YDN3Q51DVG69L2z88Go"
rangeName="Form Responses 1"
HO_cases = get_data_df(spreadSheetId, rangeName)

VMSHO_cases = HO_cases.loc[(HO_cases['Closure Product']=="VMS")]
ERPHO_cases = HO_cases.loc[(HO_cases['Closure Product']=="ERP")]

spreadSheetId = "1ptBhEQ97ThYdpf2Nnj2IqzYrXGEpTVNIszTkmEPno1Y"
wb = gc.open_by_key(spreadSheetId)

ws = wb.worksheet("VMS")
ws.clear()

# APPEND DATA TO SHEET
set_with_dataframe(ws, VMSHO_cases)

spreadSheetId = "1ptBhEQ97ThYdpf2Nnj2IqzYrXGEpTVNIszTkmEPno1Y"
wb = gc.open_by_key(spreadSheetId)

ws = wb.worksheet("ERP")
ws.clear()

# APPEND DATA TO SHEET
set_with_dataframe(ws, ERPHO_cases)

# Checker

In [ ]:
from datetime import datetime
import pytz


# Get the current time in UTC
utc_time = datetime.utcnow()


# Convert to India Standard Time (IST)
india_tz = pytz.timezone('Asia/Kolkata')
ist_time = utc_time.replace(tzinfo=pytz.utc).astimezone(india_tz)


# Format the datetime string
formatted_time = ist_time.strftime("%d-%B-%Y - %H:%M")


print(formatted_time)


import gspread
import pandas as pd
from datetime import datetime
from gspread_dataframe import set_with_dataframe


# Open the Google Sheets document
#gc = gspread.service_account(filename="path/to/your/credentials.json")
spreadsheet_id = "12jy3u_vMDIS0etAQe1DYa14mBUwXebDc9ePzgzgkTH0"
date_checker = gc.open_by_key(spreadsheet_id)


# Function to update target value
def update_target_value(target_value):
   # Find the worksheet 'scripts' and fetch data
   worksheet_scripts = date_checker.worksheet('last updated time')
   data_scripts = worksheet_scripts.get_all_values()
   headers = data_scripts[0]
   rows = data_scripts[1:]


   # Create a DataFrame with only the 'Name ' column
   df = pd.DataFrame(rows, columns=headers)


   # Find the row index for the target value
   filtered_row = df[df['Name '] == target_value]


   if not filtered_row.empty:
       # Get the index of the found row
       row_index = filtered_row.index[0]


       # Get the 'last updated time' worksheet
       worksheet_last_updated = date_checker.worksheet('last updated time')


       # Get the current datetime
       current_datetime = formatted_time


       # Update the 'last updated date / time' column with the current datetime
       worksheet_last_updated.update(f'B{row_index + 2}', [[current_datetime]])


       print(f"Current datetime {current_datetime} populated for Value {target_value}")
   else:
       print(f"No matching row found for Value {target_value}")


# Update for 'VMS Onboarding'
update_target_value('Sales Closure Script_RP')




03-March-2025 - 16:56


<ipython-input-187-db3ef831668f>:64: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet_last_updated.update(f'B{row_index + 2}', [[current_datetime]])


Current datetime 03-March-2025 - 16:56 populated for Value Sales Closure Script_RP
